In [ ]:
#Firstly select the correct graphs (100e or above) and get the data for it (in the correct form for the next function)
#Must enter in the correct order- smallest to largest

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("e ", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                a = graph.GetTitle()[42:searching.start()]
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
                
    transient = myFile.Get("TransientPropagation")
    tdetector = transient.Get("detector1")
    hole = tdetector.Get("induced_charge_h_histo")
    hy = hole.GetBinContent(7)
    y_list = []
    for i in range(2500):
        y_list.append(hole.GetBinContent(i))
    y_max = max(y_list)
    diff = (y_max - hy)/2
    
    return pulsed_array, data_x, data_y, hy, diff

#Code to firstly find the peaks of the fits for the pulses

def fit_convolve_data(x_data, y_data, time, hole_data):
    time_points_r = []
    error = []
    
    for x_pos, y_pos, h in zip(x_data, y_data, hole_data):
        if x_pos == [] and y_pos == []:
            time_points_r.append(0)
            error.append(0)
            
        else:
            for x, y in zip(x_pos, y_pos):  
                
                def func1(x, B):
                    return 0*x +B 
                
                for i in range(len(y)):
                    if np.abs(y[i]) < 0.00015 and np.abs(y[i]) != 0 and i>5:
                        hole_index = i
                        break
                    else:
                        continue
                    
                for i in range(len(y)):
                    if np.abs(y[i]) > 0.00027 and i<4:
                        electron_index = i
                        break
                    else:
                        continue
                    
                popt1, pcov1 = sc.optimize.curve_fit(func1, np.abs(list(x))[hole_index:], np.abs(list(y))[hole_index:])
                fit1 = func1(np.abs(list(x))[hole_index:], popt1[0])
                
                popt2, pcov2 = sc.optimize.curve_fit(func1, np.abs(list(x))[electron_index:hole_index-3], np.abs(list(y))[electron_index:hole_index-3])
                fit2 = func1(np.abs(list(x))[electron_index:hole_index-3], popt2[0])
                    
                time_point_raw = ((max(fit2)+min(fit2))/2)-((max(fit1)+min(fit1))/2)
                time_points_r.append(time_point_raw)
                time_error = np.sqrt(((np.diag(pcov1)[0]))+((np.diag(pcov2)[0])))
                error.append(time_error)
                
                fig = plt.figure()
                ax = fig.add_subplot(1,1,1)
                #ax.plot(np.abs(list(x))[hole_index:-5], fit1)
                ax.errorbar(np.abs(list(x))[hole_index:], fit1, yerr=np.sqrt(np.diag(pcov1)[0]), color='green', label='Hole Top-Hat Straight Line Fit')
                #ax.plot(np.abs(list(x))[electron_index:hole_index-3], fit2)
                ax.errorbar(np.abs(list(x))[electron_index:hole_index-3], fit2, yerr=np.sqrt(np.diag(pcov2)[0]), color='red', label='Electron Top-Hat Straight Line Fit')
                ax.plot(x, np.abs(y), label='Simulated Data')
                ax.set_title('Example Current Pulse for 10ps Laser and Overbiased Electric Field')
                ax.set_ylabel('Current (uA)')
                ax.set_xlabel('Time (ns)')
                ax.legend()
    
    return time_points_r, error

def getting_e_field(time_points_r, no_charge, error, position, time):
    e_field_tot_r = []
    e_field_tot_err = []
    
    v_m = 1.53*(10**9)*(300**(-0.87))
    E_c = 1.01*(300**1.55)
    B = 0.0257*(300**0.66)
        
    for n, t, p in zip(no_charge, time_points_r, position):
        print(t)
        val = t*(10**(-4))/np.abs(int(float(n[0])))
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((500000/E_c)**B))**(1/B)))
        e_field_r = val/(charge*weighting_field*mu_si)
        e_field_tot_r.append(e_field_r)
    
    for n, t, p in zip(no_charge, error, position):
        val = t*(10**(-4))/np.abs(int(float(n[0])))
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((500000/E_c)**B))**(1/B)))
        e_field_e = val/(charge*weighting_field*mu_si)
        e_field_tot_err.append(e_field_e)

    def func1(x, A, B):
        return A*x + B
    
    poptr, pcovr = sc.optimize.curve_fit(func1, position, e_field_tot_r)
    fit2 = func1(np.array(position), poptr[0], poptr[1])
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, [500000, 500000, 500000,500000, 500000, 500000,500000, 500000], 'black', label='Allpix Simulated Electric Field')
     
    #ax.errorbar(position, fit1, yerr=np.sqrt(np.diag(pcov)[1]), label='Straight Line Fit for Convolution Data', color='purple')
    #ax.plot(position, e_field_tot_c, 'green', label=' Convolution Electric Field Data')
    print(e_field_tot_err)
    ax.errorbar(position, e_field_tot_r, yerr=e_field_tot_err, label='E_Field From Simulated Current Pulses', color='blue')
    #ax.plot(position, e_field_tot_r, 'red', label='Electric Field Data')
    #ax.plot(np.linspace(5, 45, 5), [100000,100000,100000,100000,100000])
    ax.set_title(f'Electric Field against Position for a Single Electron Pulse')
    ax.set_ylabel('Electric Field (V/cm)')
    ax.set_xlabel('Position (\u03bcm)')
    ax.set_ylim(450000, 550000)
    ax.legend()

 
files = ["one_e_n40.root", "one_e_n35.root", "one_e_n30.root", "one_e_n25.root", "one_e_n20.root", "one_e_n15.root", "one_e_n10.root", "one_e_n5.root",]
x_values, y_values, pulse_names, hole_dat = [], [], [], []
for dat in files:
    pulse, x_dat, y_dat, hole_data, diff = collect_pulse_data(dat)
    pulse_names.append(pulse)
    x_values.append(x_dat)
    y_values.append(y_dat)
    hole_dat.append(hole_data)

time_points_r, error = fit_convolve_data(x_values, y_values, 0.08, hole_dat) 
getting_e_field(time_points_r, pulse_names, error, [40, 35, 30, 25, 20, 15, 10, 5], 0.08)

In [ ]:
#Firstly select the correct graphs (100e or above) and get the data for it (in the correct form for the next function)
#Must enter in the correct order- smallest to largest

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("ke", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                a = graph.GetTitle()[42:searching.start()]
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
                
    transient = myFile.Get("TransientPropagation")
    tdetector = transient.Get("detector1")
    electron = tdetector.Get("induced_charge_e_histo")
    ey = np.abs(electron.GetBinContent(7))
    print(ey)
    y_list = []
    for i in range(2500):
        y_list.append(electron.GetBinContent(i))
    y_max = max(np.abs(y_list))
    diff = (y_max - ey)/2
    
    return pulsed_array, data_x, data_y, ey, diff

#Code to firstly find the peaks of the fits for the pulses

def fit_convolve_data(x_data, y_data, time, hole_data):
    max_points_r = []
    time_points_r = []
    max_points_c = []
    time_points_c = []
    error = []
    
    for x_pos, y_pos, h in zip(x_data, y_data, hole_data):
        if x_pos == [] and y_pos == []:
            max_points_r.append(0)
            time_points_r.append(0)
            max_points_c.append(0)
            time_points_c.append(0)
            
        else:
            for x, y in zip(x_pos, y_pos):  

                dx = x[1]-x[0]
                x_pdf = np.arange(x[0], x[-1], dx)        
                y_pdf = sc.stats.norm.pdf(x_pdf, 0.1, 0.075) 
                y_convolve = sc.signal.fftconvolve(np.abs(y), y_pdf, mode='full')*dx
                x_convolve = np.linspace(0, x[-1], len(y_convolve))
                
                #hole_cont = ((h*sc.constants.e)/(0.01*(10**(-9))))*(10**6)
                    
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= 3 and x_convolve[i-1] < 3:
                        hole_index_c = i
                        break
                    else:
                        continue
                    
                for i in range(len(x)):
                    if x[i] >= 2.6 and x[i-1] < 2.6:
                        hole_index_r = i
                        break
                    else:
                        continue
                    
                hole_cont_c = np.abs(y_convolve[hole_index_c])
                hole_cont = np.abs(y[hole_index_r])
                
                fig = plt.figure()
                ax = fig.add_subplot(1,1,1)
                #ax.plot(x_convolve, y_convolve, label='Convolution')
                ax.plot(x, np.abs(y), label='Simulated Data')
                #ax.plot(x_pdf, y_pdf)
                ax.set_title('Example Current Pulse for 10ps Laser and Overbiased Electric Field')
                ax.set_ylabel('Current (\u03bcA)')
                ax.set_xlabel('Time (ns)')
                ax.legend()
                
                max_point_raw = max(np.abs(y))-hole_cont
                max_points_r.append(max_point_raw)
                
                max_point_con = max(y_convolve)
                index_max = np.where(y_convolve==max_point_con)
                max_points_c.append(max_point_con)
                
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= time and x_convolve[i-1] < time:
                        time_index_c = i
                        break
                    else:
                        continue
                    
                for i in range(len(x)):
                    if x[i] >= time and x[i-1] < time:
                        time_index_r = i
                        break
                    else:
                        continue
                    
                time_point_raw = np.abs(y[time_index_r])-hole_cont
                time_points_r.append(time_point_raw)
            
                time_point_con = np.abs(y_convolve[time_index_c])-hole_cont_c
                time_points_c.append(time_point_con)
                
                errors = (max(np.abs(y))-np.abs(y[time_index_r]))/2
                error.append(errors)

        
    return max_points_r, time_points_r, max_points_c, time_points_c, error

def getting_e_field(time_points_r, time_points_c, no_charge, error, error2, position, time):
    e_field_tot_r = []
    e_field_tot_c = []
    e_field_tot_err = []
    e_field_tot_err2 = []
    
    v_m = 1.53*(10**9)*(300**(-0.87))
    E_c = 1.01*(300**1.55)
    B = 0.0257*(300**0.66)
        
    for n, t, p in zip(no_charge, time_points_c, position):
        val = t*(10**(-4))/np.abs(int(float(n[0])*1000))
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((5000/E_c)**B))**(1/B)))
        e_field_c = val/(charge*weighting_field*mu_si)
        e_field_tot_c.append(e_field_c)
        
    for n, t, p in zip(no_charge, time_points_r, position):
        val = t*(10**(-4))/np.abs(int(float(n[0])*1000))
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((5000/E_c)**B))**(1/B)))
        e_field_r = val/(charge*weighting_field*mu_si)
        e_field_tot_r.append(e_field_r)
        
    for n, t, p in zip(no_charge, error, position):
        val = t*(10**(-4))/np.abs(int(float(n[0])*1000))
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((5000/E_c)**B))**(1/B)))
        e_field_e = val/(charge*weighting_field*mu_si)
        e_field_tot_err.append(e_field_e)
        
    for n, t, p in zip(no_charge, error2, position):
        val = t*(10**(-4))/np.abs(int(float(n[0])*1000))
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((5000/E_c)**B))**(1/B)))
        e_field_e2 = val/(charge*weighting_field*mu_si)
        e_field_tot_err2.append(e_field_e2)

    def func1(x, A, B):
        return A*x + B
    
    popt, pcov = sc.optimize.curve_fit(func1, position, e_field_tot_c)
    fit1 = func1(np.array(position), popt[0], popt[1])
    
    poptr, pcovr = sc.optimize.curve_fit(func1, position, e_field_tot_r)
    fit2 = func1(np.array(position), poptr[0], poptr[1])
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000], 'black', label='Allpix Simulated Electric Field')
     
    #ax.errorbar(position, fit1, yerr=np.sqrt(np.diag(pcov)[1]), label='Straight Line Fit for Convolution Data', color='purple')
    #ax.plot(position, e_field_tot_c, 'green', label=' Convolution Electric Field Data')
    
    #ax.errorbar(position, fit2, yerr=np.sqrt(np.diag(pcovr)[1]), label='Straight Line Fit for Electric Field Data', color='blue')
    #ax.plot(position, e_field_tot_r, 'red', label='Electric Field Data')
    
    #ax.plot(position[2:10], fit2, 'g--', label='Straight Line Fit for Real Data')
    ax.errorbar(position, e_field_tot_r, yerr=e_field_tot_err, label='E_Field From Simulated Current Pulses (holes and electrons)', color='blue')
    #ax.plot(position, e_field_tot_r, 'red', label='Electric Field Data')
    
    ax.errorbar(position, e_field_tot_c, yerr=e_field_tot_err2, label='E_Field From Simulated Charge Pulses (electrons only)', color='orangered')
    #ax.plot(position, e_field_tot_c, 'green', label='Electric Field Data')
    ax.set_ylim(4400, 5600)
    ax.set_title(f'Electric Field against Position of the Laser')
    ax.set_ylabel('Electric Field (V/cm)')
    ax.set_xlabel('Position (electrodes at \u00B150) (\u03bcm)')
    ax.set_ylim(4000, 6200)
    ax.legend()

 
files = ["ppd_n40.root", "ppd_n35.root", "ppd_n30.root", "ppd_n25.root","ppd_n20.root", "ppd_n15.root", "ppd_n10.root", "ppd_n05.root"]
x_values, y_values, pulse_names, e_data, error2 = [], [], [], [], []
for dat in files:
    pulse, x_dat, y_dat, electron_data, diff = collect_pulse_data(dat)
    pulse_names.append(pulse)
    x_values.append(x_dat)
    y_values.append(y_dat)
    e_dat = ((electron_data*sc.constants.e)/(0.01*(10**(-9))))*(10**6)
    e_data.append(e_dat)
    error2.append(((diff*sc.constants.e)/(0.01*(10**(-9))))*(10**6))
    
print(e_data)

max_points_r, time_points_r, max_points_c, time_points_c, error = fit_convolve_data(x_values, y_values, 0.15, e_data) 
getting_e_field(time_points_r, e_data, pulse_names, error, error2, [40, 35, 30, 25, 20, 15, 10, 5], 0.15)

In [ ]:
#Firstly select the correct graphs (100e or above) and get the data for it (in the correct form for the next function)
#Must enter in the correct order- smallest to largest

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("ke", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                a = graph.GetTitle()[42:searching.start()]
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
                
    transient = myFile.Get("TransientPropagation")
    tdetector = transient.Get("detector1")
    electron = tdetector.Get("induced_charge_e_histo")
    ey = np.abs(electron.GetBinContent(7))
    print(ey)
    y_list = []
    for i in range(2500):
        y_list.append(electron.GetBinContent(i))
    y_max = max(np.abs(y_list))
    diff = (y_max - ey)/2
    
    return pulsed_array, data_x, data_y, ey, diff

#Code to firstly find the peaks of the fits for the pulses

def fit_convolve_data(x_data, y_data, time, hole_data):
    max_points_r = []
    time_points_r = []
    max_points_c = []
    time_points_c = []
    error = []
    
    for x_pos, y_pos, h in zip(x_data, y_data, hole_data):
        if x_pos == [] and y_pos == []:
            max_points_r.append(0)
            time_points_r.append(0)
            max_points_c.append(0)
            time_points_c.append(0)
            
        else:
            for x, y in zip(x_pos, y_pos):  

                dx = x[1]-x[0]
                x_pdf = np.arange(x[0], x[-1], dx)        
                y_pdf = sc.stats.norm.pdf(x_pdf, 0, 1) 
                y_convolve = sc.signal.fftconvolve(np.abs(y), y_pdf, mode='full')*dx
                x_convolve = np.linspace(0, x[-1], len(y_convolve))
                
                #hole_cont = ((h*sc.constants.e)/(0.01*(10**(-9))))*(10**6)
                    
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= 1 and x_convolve[i-1] < 1:
                        hole_index_c = i
                        break
                    else:
                        continue
                    
                for i in range(len(x)):
                    if x[i] >= 2.6 and x[i-1] < 2.6:
                        hole_index_r = i
                        break
                    else:
                        continue
                    
                hole_cont_c = np.abs(y_convolve[hole_index_c])
                hole_cont = np.abs(y[hole_index_r])
                
                fig = plt.figure()
                ax = fig.add_subplot(1,1,1)
                ax.plot(x_convolve, y_convolve, label='Convolution of Simulated Data')
                ax.plot(x, np.abs(y), label='Simulated Data')
                #ax.plot(x_pdf, y_pdf)
                ax.set_title('Example Current Pulse for 10ps Laser and Overbiased Electric Field')
                ax.set_ylabel('Current (\u03bcA)')
                ax.set_xlabel('Time (ns)')
                ax.legend()
                
                max_point_raw = max(np.abs(y))-hole_cont
                max_points_r.append(max_point_raw)
                
                max_point_con = max(y_convolve)
                index_max = np.where(y_convolve==max_point_con)
                max_points_c.append(max_point_con)
                
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= time and x_convolve[i-1] < time:
                        time_index_c = i
                        break
                    else:
                        continue
                    
                for i in range(len(x)):
                    if x[i] >= time and x[i-1] < time:
                        time_index_r = i
                        break
                    else:
                        continue
                    
                time_point_raw = np.abs(y[time_index_r])-hole_cont
                time_points_r.append(time_point_raw)
            
                time_point_con = np.abs(y_convolve[time_index_c])#-hole_cont_c
                time_points_c.append(time_point_con)
                
                errors = (max(np.abs(y))-np.abs(y[time_index_r]))/2
                error.append(errors)

        
    return max_points_r, time_points_r, max_points_c, time_points_c, error

def getting_e_field(time_points_r, time_points_c, no_charge, error, error2, position, time):
    e_field_tot_r = []
    e_field_tot_c = []
    e_field_tot_err = []
    e_field_tot_err2 = []
    
    v_m = 1.53*(10**9)*(300**(-0.87))
    E_c = 1.01*(300**1.55)
    B = 0.0257*(300**0.66)
        
    for n, t, p in zip(no_charge, time_points_c, position):
        val = t*(10**(-4))/np.abs(int(float(n[0])*1000))
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((5000/E_c)**B))**(1/B)))
        e_field_c = val/(charge*weighting_field*mu_si)
        e_field_tot_c.append(e_field_c)
        
    for n, t, p in zip(no_charge, time_points_r, position):
        val = t*(10**(-4))/np.abs(int(float(n[0])*1000))
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((5000/E_c)**B))**(1/B)))
        e_field_r = val/(charge*weighting_field*mu_si)
        e_field_tot_r.append(e_field_r)
        
    for n, t, p in zip(no_charge, error, position):
        val = t*(10**(-4))/np.abs(int(float(n[0])*1000))
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((5000/E_c)**B))**(1/B)))
        e_field_e = val/(charge*weighting_field*mu_si)
        e_field_tot_err.append(e_field_e)
        
    for n, t, p in zip(no_charge, error, position):
        val = t*(10**(-4))/np.abs(int(float(n[0])*1000))
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((5000/E_c)**B))**(1/B)))
        e_field_e2 = val/(charge*weighting_field*mu_si)
        e_field_tot_err2.append(e_field_e2)

    def func1(x, A, B):
        return A*x + B
    
    popt, pcov = sc.optimize.curve_fit(func1, position, e_field_tot_c)
    fit1 = func1(np.array(position), popt[0], popt[1])
    
    poptr, pcovr = sc.optimize.curve_fit(func1, position, e_field_tot_r)
    fit2 = func1(np.array(position), poptr[0], poptr[1])
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000], 'black', label='Allpix Simulated Electric Field')
     
    #ax.errorbar(position, fit1, yerr=np.sqrt(np.diag(pcov)[1]), label='Straight Line Fit for Convolution Data', color='purple')
    #ax.plot(position, e_field_tot_c, 'green', label=' Convolution Electric Field Data')
    
    #ax.errorbar(position, fit2, yerr=np.sqrt(np.diag(pcovr)[1]), label='Straight Line Fit for Electric Field Data', color='blue')
    #ax.plot(position, e_field_tot_r, 'red', label='Electric Field Data')
    
    #ax.plot(position[2:10], fit2, 'g--', label='Straight Line Fit for Real Data')
    #ax.errorbar(position, e_field_tot_r, yerr=e_field_tot_err, label='Electric Field From Simulated Current Pulses (holes and electrons)', color='blue')
    #ax.plot(position, e_field_tot_r, 'red', label='Electric Field Data')
    
    ax.errorbar(position, e_field_tot_c, yerr=e_field_tot_err2, label='Electric Field From Simulated Charge Pulses (electrons only)', color='purple')
    #ax.plot(position, e_field_tot_c, 'green', label='Electric Field Data')
    #ax.set_ylim(4400, 5600)
    ax.set_title(f'Electric Field against Position of the Laser')
    ax.set_ylabel('Electric Field (V/cm)')
    ax.set_xlabel('Position (electrodes at \u00B150) (um)')
    ax.legend()

 
files = ["ppd_n40.root", "ppd_n35.root", "ppd_n30.root", "ppd_n25.root","ppd_n20.root", "ppd_n15.root", "ppd_n10.root", "ppd_n05.root"]
x_values, y_values, pulse_names, e_data, error2 = [], [], [], [], []
for dat in files:
    pulse, x_dat, y_dat, electron_data, diff = collect_pulse_data(dat)
    pulse_names.append(pulse)
    x_values.append(x_dat)
    y_values.append(y_dat)
    e_dat = ((electron_data*sc.constants.e)/(0.01*(10**(-9))))*(10**6)
    e_data.append(e_dat)
    error2.append(((diff*sc.constants.e)/(0.01*(10**(-9))))*(10**6))
    
print(e_data)

max_points_r, time_points_r, max_points_c, time_points_c, error = fit_convolve_data(x_values, y_values, 0.12, e_data) 
getting_e_field(time_points_r, time_points_c, pulse_names, error, error2, [40, 35, 30, 25, 20, 15, 10, 5], 0.12)

In [ ]:
#Firstly select the correct graphs (100e or above) and get the data for it (in the correct form for the next function)
#Must enter in the correct order- smallest to largest

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("current", graph.GetTitle())
            searching1 = re.search(" = ", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                a = graph.GetTitle()[searching1.end():searching1.end()+4]
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
                
    transient = myFile.Get("TransientPropagation")
    tdetector = transient.Get("detector1")
    hole = tdetector.Get("induced_charge_h_histo")
    hy = hole.GetBinContent(7)
    y_list = []
    for i in range(2500):
        y_list.append(hole.GetBinContent(i))
    y_max = max(np.abs(y_list))
    diff = (y_max - hy)/2
    
    return pulsed_array, data_x, data_y, hy, diff

def get_electrons(file):
    myFile = TFile.Open(file, "read")
    transient = myFile.Get("TransientPropagation")
    tdetector = transient.Get("detector1")
    electron = tdetector.Get("induced_charge_e_histo")
    no_bins = electron.GetNbinsX()
    y_list = []
    for i in range(no_bins):
        y_list.append(electron.GetBinContent(i))
    y_max = max(np.abs(y_list))
    y_max_correct = ((y_max*sc.constants.e)/(0.01*(10**(-9))))*(10**6)
    
    return y_max_correct
    

#Code to firstly find the peaks of the fits for the pulses

def fit_convolve_data(x_data, y_data, time, hole_data):
    max_points_r = []
    time_points_r = []
    max_points_c = []
    time_points_c = []
    error = []
    
    for x_pos, y_pos, h in zip(x_data, y_data, hole_data):
        if x_pos == [] and y_pos == []:
            max_points_r.append(0)
            time_points_r.append(0)
            max_points_c.append(0)
            time_points_c.append(0)
            
        else:
            for x, y in zip(x_pos, y_pos):  

                dx = x[1]-x[0]
                x_pdf = np.arange(x[0], x[-1], dx)        
                y_pdf = sc.stats.norm.pdf(x_pdf, 0.1, 0.075) 
                y_convolve = sc.signal.fftconvolve(np.abs(y), y_pdf, mode='full')*dx
                x_convolve = np.linspace(0, x[-1], len(y_convolve))
                
                hole_cont = np.abs(((h*sc.constants.e)/(0.01*(10**(-9))))*(10**6))
                    
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= 0.35 and x_convolve[i-1] < 0.35:
                        hole_index_c = i
                        break
                    else:
                        continue
                    
                def gauss(x,a,x0,sigma):
                    return a*np.exp(-(x-x0)**2/(2*sigma**2))

                x1 = np.array(x)
                y1 = np.array(y)
                n = len(x1)                         
                mean = sum(x1*y1)/n                   
                sigma = sum(y1*(x1-mean)**2)/n
                popt, pcov = sc.optimize.curve_fit(gauss, x, y, p0=[1,mean,sigma])
                y_fit = gauss(x1, popt[0], popt[1], popt[2])
                
                error.append(np.sqrt(np.diag(pcov)[0]))
                print(pcov[0])
                    
                hole_cont_c = np.abs(y_convolve[hole_index_c])
    
                
                fig = plt.figure()
                ax = fig.add_subplot(1,1,1)
                #ax.plot(x_convolve, y_convolve, label='Convolution')
                ax.plot(x, y, label='Simulated Data')
                #ax.errorbar(np.abs(list(x))[:max_index-3], fit1, yerr=np.sqrt(np.diag(pcov1)[0]), color='purple')
                ax.plot(x, y_fit, label='Gaussian Fit for Simulated Data')
                ax.set_title('Example Current Pulse for 10ps Laser and PIN Diode E_Field')
                ax.set_ylabel('Current (\u03bcA)')
                ax.set_xlabel('Time (ns)')
                ax.legend()
                
                max_point_raw = 0.75*max(np.abs(y_fit))-hole_cont
                max_points_r.append(max_point_raw)
                
                max_point_con = max(y_convolve)
                index_max = np.where(y_convolve==max_point_con)
                max_points_c.append(max_point_con)
                
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= time and x_convolve[i-1] < time:
                        time_index_c = i
                        break
                    else:
                        continue
                    
                for i in range(len(x)):
                    if x[i] >= time and x[i-1] < time:
                        time_index_r = i
                        break
                    else:
                        continue
                    
                time_point_raw = np.abs(y_fit[time_index_r])-hole_cont
                time_points_r.append(time_point_raw)
                
                time_point_con = np.abs(y_convolve[time_index_c])-hole_cont_c
                time_points_c.append(time_point_con)
        
    return max_points_r, time_points_r, max_points_c, time_points_c, error

def getting_e_field(time_points_r, electron_data, no_charge, error, position, time, file):
    e_field_tot_r = []
    e_field_error = []
    
    v_m = 1.53*(10**9)*(300**(-0.87))
    E_c = 1.01*(300**1.55)
    B = 0.0257*(300**0.66)
    
    def mu_e_field(p, f):
        myFile = TFile.Open(f, "read")
        data = myFile.Get("ElectricFieldReader")
        detector = data.Get("detector1")
        efield = detector.Get("field1d_z")
        no_bins = efield.GetNbinsX()
        bin_size = 0.72/no_bins
        
        pos = p*10**(-3)
        placement = -0.36
        for bin in range(no_bins):  
            placement += bin_size
            if placement >= pos:
                field_strength = efield.GetBinContent(bin+1)
                break
            
        return field_strength
    
    def weighting_potential(f):
        myFile = TFile.Open(f, "read")
        data = myFile.Get("WeightingPotentialReader")
        detector = data.Get("detector1")
        wfield = detector.Get("potential1d")
        no_bins = wfield.GetNbinsX()
        bin_size = 0.72/no_bins
        w_tot = []
        grad = []
        point = []
        count1 = 0
        for i in range(no_bins):
            w_tot.append(wfield.GetBinContent(i+1))
            count1 +=1
            if wfield.GetBinContent(i) != wfield.GetBinContent(i-1):
                point.append(((count1*bin_size)-0.36)*10**(-3))
                grad.append(wfield.GetBinContent(i))
        
        calc_field = np.gradient(np.interp(np.linspace(-0.36*10**(-3), 0.36*10**(-3), int(no_bins/2)), point, grad), (0.72*10**(-3))/(int(no_bins/2)))
        
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
        ax.plot(np.linspace(-0.36*10**(-3), 0.36*10**(-3), int(no_bins/2)), np.interp(np.linspace(-0.36*10**(-3), 0.36*10**(-3), int(no_bins/2)), point, grad), (0.72*10**(-3))/int(no_bins/2))
        
        pos = p*10**(-3)
        placement = -0.36
        for bin in range(int(no_bins/2)):  
            placement += ((0.72)/(int(no_bins/2)))
            if placement >= pos:
                wpos = calc_field[bin]
                break
            
        #plt.figure()
        #ax.plot(np.linspace(-0.36*10**(-3), 0.36*10**(-3), no_bins), calc_field)
        wf = wpos/((0.36-placement)*10**(-3))
        #print(wpos, placement, wf)
        return wpos
        
    for n, t, p in zip(no_charge, time_points_r, position):
        val = t*(10**(-4))/int(float(n[0])*1000)
        charge = 1.6*(10**(-19))
        weighting_field = weighting_potential(file)
        mu_si = (v_m/E_c)*(1/((1+((np.abs(mu_e_field(p, file))/E_c)**B))**(1/B)))
        e_field_r = val/(charge*weighting_field*mu_si)
        e_field_tot_r.append(e_field_r)
        print(t, weighting_field, e_field_r, n)
        
    for n, t, p in zip(no_charge, error, position):
        val = t*(10**(-4))/int(float(n[0])*1000)
        charge = 1.6*(10**(-19))
        weighting_field = weighting_potential(file)
        mu_si = (v_m/E_c)*(1/((1+((np.abs(mu_e_field(p, file))/E_c)**B))**(1/B)))
        e_field_e = val/(charge*weighting_field*mu_si)
        e_field_error.append(e_field_e)
    
    x_efield = np.linspace(position[0], position[-1], 10000)
    y_efield = []
    for i in x_efield:
        y_dat = mu_e_field(i, file)
        y_efield.append(y_dat)
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(x_efield, y_efield, 'black', label='TCAD Simulated Electric Field')
    
    #ax.errorbar(position, fit2, yerr=np.sqrt(np.diag(pcovr)[1]), label='Straight Line Fit for Electric Field Data', color='blue')
    ax.plot(position, e_field_tot_r, 'red', label='Electric Field Data')
    #ax.plot(position, e_field_electron, 'green', label='Electric Field from Electron Count')
    #ax.errorbar(position, e_field_tot_r, yerr=e_field_error, color='blue', label='E_Field from Simulated Current Pulses')
    ax.set_title(f'Electric Field against Position at Maximum Point of Gaussian Fit')
    ax.set_ylabel('Electric Field (V/cm)')
    ax.set_xlabel('Position (\u03bcm)')
    #ax.set_ylim(-2000, 11000)
    ax.legend()

 
files= ['root_files/tcad_efield1_-350.root', 'root_files/tcad_efield1_-340.root', 'root_files/tcad_efield1_-330.root', 'root_files/tcad_efield1_-320.root', 'root_files/tcad_efield1_-310.root', 'root_files/tcad_efield1_-300.root', 'root_files/tcad_efield1_-290.root', 'root_files/tcad_efield1_-280.root', 'root_files/tcad_efield1_-270.root', 'root_files/tcad_efield1_-260.root', 'root_files/tcad_efield1_-250.root', 'root_files/tcad_efield1_-240.root', 'root_files/tcad_efield1_-230.root', 'root_files/tcad_efield1_-220.root', 'root_files/tcad_efield1_-210.root', 'root_files/tcad_efield1_-200.root', 'root_files/tcad_efield1_-175.root', 'root_files/tcad_efield1_-150.root']
x_values, y_values, pulse_names, hole_dat = [], [], [], []
for dat in files:
    pulse, x_dat, y_dat, hole_data, diff = collect_pulse_data(dat)
    pulse_names.append(pulse)
    x_values.append(x_dat)
    y_values.append(y_dat)
    hole_dat.append(hole_data)

electron_data = []
for e in files:
    electron_val = get_electrons(e)
    electron_data.append(electron_val)

max_points_r, time_points_r, max_points_c, time_points_c, error1 = fit_convolve_data(x_values, y_values, 0.5, hole_dat) 
getting_e_field(time_points_r, electron_data, pulse_names, error1, [350, 340, 330, 320, 310, 300, 290, 280, 270, 260, 250, 240, 230, 220, 210, 200, 175, 150], 0.04, "root_files/tcad_efield1_330.root")

In [ ]:
#Firstly select the correct graphs (100e or above) and get the data for it (in the correct form for the next function)
#Must enter in the correct order- smallest to largest

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("current", graph.GetTitle())
            searching1 = re.search(" = ", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                a = graph.GetTitle()[searching1.end():searching1.end()+4]
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
                
    transient = myFile.Get("TransientPropagation")
    tdetector = transient.Get("detector1")
    hole = tdetector.Get("induced_charge_h_histo")
    hy = np.abs(hole.GetBinContent(7))
    y_list = []
    for i in range(2500):
        y_list.append(hole.GetBinContent(i))
    y_max = max(np.abs(y_list))
    diff = (y_max - hy)/2
    
    return pulsed_array, data_x, data_y, y_max, diff

def get_electrons(file):
    myFile = TFile.Open(file, "read")
    transient = myFile.Get("TransientPropagation")
    tdetector = transient.Get("detector1")
    electron = tdetector.Get("induced_charge_e_histo")
    no_bins = electron.GetNbinsX()
    y_list = []
    for i in range(no_bins):
        y_list.append(electron.GetBinContent(i))
    y_max = max(np.abs(y_list))
    y_max_correct = ((y_max*sc.constants.e)/(0.01*(10**(-9))))*(10**6)
    
    return y_max_correct
    

#Code to firstly find the peaks of the fits for the pulses

def fit_convolve_data(x_data, y_data, time, hole_data):
    max_points_r = []
    time_points_r = []
    max_points_c = []
    time_points_c = []
    
    for x_pos, y_pos, h in zip(x_data, y_data, hole_data):
        if x_pos == [] and y_pos == []:
            max_points_r.append(0)
            time_points_r.append(0)
            max_points_c.append(0)
            time_points_c.append(0)
            
        else:
            for x, y in zip(x_pos, y_pos):  

                dx = x[1]-x[0]
                x_pdf = np.arange(x[0], x[-1], dx)        
                y_pdf = sc.stats.norm.pdf(x_pdf, 0.1, 0.075) 
                y_convolve = sc.signal.fftconvolve(np.abs(y), y_pdf, mode='full')*dx
                x_convolve = np.linspace(0, x[-1], len(y_convolve))
                
                hole_cont = ((h*sc.constants.e)/(0.01*(10**(-9))))*(10**6)
                    
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= 0.35 and x_convolve[i-1] < 0.35:
                        hole_index_c = i
                        break
                    else:
                        continue
                    
                max_index = (list(y)).index(min(y))
                
                def func1(x, a, b):
                   return a*np.exp(b*x)
                
                popt, pcov = sc.optimize.curve_fit(func1, np.abs(list(x))[:max_index+3], list(y)[:max_index+3])
                y_fit = func1(np.abs(list(x)[:max_index+3]), popt[0], popt[1])
                    
                hole_cont_c = np.abs(y_convolve[hole_index_c])
                
                fig = plt.figure()
                ax = fig.add_subplot(1,1,1)
                #ax.plot(x_convolve, y_convolve, label='Convolution')
                ax.plot(x, y, label='Real Data')
                ax.errorbar(np.abs(list(x))[:max_index+3], y_fit, yerr=np.sqrt(np.diag(pcov)[0]), color='purple')
                #ax.plot(x_pdf, y_pdf)
                ax.set_title('Example Current Pulse for 10ps Laser and Overbiased Electric Field')
                ax.set_ylabel('Current (uA)')
                ax.set_xlabel('Time (ns)')
                ax.legend()
                
                max_point_raw = max(np.abs(y))-0.03
                max_points_r.append(max_point_raw)
                
                max_point_con = max(y_convolve)
                index_max = np.where(y_convolve==max_point_con)
                max_points_c.append(max_point_con)
                
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= time and x_convolve[i-1] < time:
                        time_index_c = i
                        break
                    else:
                        continue
                    
                for i in range(len(x)):
                    if x[i] >= time and x[i-1] < time:
                        time_index_r = i
                        break
                    else:
                        continue
                    
                time_point_raw = np.abs(y[time_index_r])-hole_cont
                time_points_r.append(time_point_raw)
                
                time_point_con = np.abs(y_convolve[time_index_c])-hole_cont_c
                time_points_c.append(time_point_con)
        
    return max_points_r, time_points_r, max_points_c, time_points_c

def getting_e_field(time_points_r, electron_data, no_charge, error, position, time, file):
    e_field_tot_r = []
    e_field_electron = []
    
    v_m = 1.53*(10**9)*(300**(-0.87))
    E_c = 1.01*(300**1.55)
    B = 0.0257*(300**0.66)
    
    def mu_e_field(p, f):
        myFile = TFile.Open(f, "read")
        data = myFile.Get("ElectricFieldReader")
        detector = data.Get("detector1")
        efield = detector.Get("field1d_z")
        no_bins = efield.GetNbinsX()
        bin_size = 0.72/no_bins
        
        pos = p*10**(-3)
        placement = -0.36
        for bin in range(no_bins):  
            placement += bin_size
            if placement >= pos:
                field_strength = efield.GetBinContent(bin+1)
                break
            
        return field_strength
    
    def weighting_potential(f):
        myFile = TFile.Open(f, "read")
        data = myFile.Get("WeightingPotentialReader")
        detector = data.Get("detector1")
        wfield = detector.Get("potential1d")
        no_bins = wfield.GetNbinsX()
        bin_size = 0.72/no_bins
        w_tot = []
        for i in range(no_bins):
            w_tot.append(wfield.GetBinContent(i))
        calc_field = np.gradient(w_tot, (bin_size*10**(-3)))
        pos = p*10**(-3)
        placement = -0.36
        for bin in range(no_bins):  
            placement += bin_size
            if placement >= pos:
                wf = calc_field[bin]
                break
        return wf
        
    for n, t, p in zip(no_charge, time_points_r, position):
        print(int(float(n[0])))
        if np.abs(int(float(n[0])))>100:
            fixed_n = float(n[0])/1000
        else:
            fixed_n = float(n[0])
        val = t*(10**(-4))/int(fixed_n*1000)
        charge = 1.6*(10**(-19))
        weighting_field = weighting_potential(file)
        mu_si = (v_m/E_c)*(1/((1+((np.abs(mu_e_field(p, file))/E_c)**B))**(1/B)))
        e_field_r = val/(charge*weighting_field*mu_si)
        e_field_tot_r.append(e_field_r)
        print(t, weighting_field, e_field_r, mu_si, n)
        
    for n, t, p in zip(no_charge, electron_data, position):
        val = t*(10**(-4))/int(float(n[0])*1000)
        charge = 1.6*(10**(-19))
        weighting_field = weighting_potential(file)
        mu_si = (v_m/E_c)*(1/((1+((np.abs(mu_e_field(p, file))/E_c)**B))**(1/B)))
        e_field_e = val/(charge*weighting_field*mu_si)
        e_field_electron.append(e_field_e)
    
    x_efield = np.linspace(position[0], position[-1], 10000)
    y_efield = []
    for i in x_efield:
        y_dat = mu_e_field(i, file)
        y_efield.append(y_dat)
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(x_efield, np.abs(y_efield), 'black', label='Actual Electric Field')
    
    #ax.errorbar(position, fit2, yerr=np.sqrt(np.diag(pcovr)[1]), label='Straight Line Fit for Electric Field Data', color='blue')
    ax.plot(position, e_field_tot_r, 'red', label='Electric Field Data')
    
    #ax.plot(position, e_field_electron, 'green', label='Electric Field from Electron Count')
    
    ax.set_title(f'Electric Field against Position at t={time}ns')
    ax.set_ylabel('Electric Field (V/cm)')
    ax.set_xlabel('Position (um)')
    #ax.set_ylim(100000, 300000)
    ax.legend()

 
files= ['root_files/tcad_efield1_pad_-350.root', 'root_files/tcad_efield1_pad_-340.root', 'root_files/tcad_efield1_pad_-330.root', 'root_files/tcad_efield1_pad_-320.root', 'root_files/tcad_efield1_pad_-310.root', 'root_files/tcad_efield1_pad_-300.root', 'root_files/tcad_efield1_pad_-290.root', 'root_files/tcad_efield1_pad_-280.root', 'root_files/tcad_efield1_pad_-270.root', 'root_files/tcad_efield1_pad_-260.root', 'root_files/tcad_efield1_pad_-250.root', 'root_files/tcad_efield1_pad_-240.root', 'root_files/tcad_efield1_pad_-230.root', 'root_files/tcad_efield1_pad_-220.root', 'root_files/tcad_efield1_pad_-210.root', 'root_files/tcad_efield1_pad_-200.root', 'root_files/tcad_efield1_pad_-175.root', 'root_files/tcad_efield1_pad_-150.root']
x_values, y_values, pulse_names, hole_dat, error = [], [], [], [], []
for dat in files:
    pulse, x_dat, y_dat, hole_data, diff = collect_pulse_data(dat)
    pulse_names.append(pulse)
    x_values.append(x_dat)
    y_values.append(y_dat)
    hole_dat.append(hole_data)
    error.append((((diff*sc.constants.e)/(0.01*(10**(-9))))*(10**6)))

electron_data = []
for e in files:
    electron_val = get_electrons(e)
    electron_data.append(electron_val)

max_points_r, time_points_r, max_points_c, time_points_c = fit_convolve_data(x_values, y_values, 0.75, hole_dat) 
getting_e_field(time_points_r, electron_data, pulse_names, error, [350, 340, 330, 320, 310, 300, 290, 280, 270, 260, 250, 240, 230, 220, 210, 200, 175, 150], 0.04, 'root_files/tcad_efield1_pad_-340.root')

In [ ]:
#Firstly select the correct graphs (100e or above) and get the data for it (in the correct form for the next function)
#Must enter in the correct order- smallest to largest

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("current", graph.GetTitle())
            searching1 = re.search(" = ", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                a = graph.GetTitle()[searching1.end():searching1.end()+4]
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
                
    transient = myFile.Get("TransientPropagation")
    tdetector = transient.Get("detector1")
    hole = tdetector.Get("induced_charge_h_histo")
    hy = hole.GetBinContent(7)
    y_list = []
    for i in range(2500):
        y_list.append(hole.GetBinContent(i))
    y_max = max(np.abs(y_list))
    diff = (y_max - hy)/2
    
    return pulsed_array, data_x, data_y, hy, diff

def get_electrons(file):
    myFile = TFile.Open(file, "read")
    transient = myFile.Get("TransientPropagation")
    tdetector = transient.Get("detector1")
    electron = tdetector.Get("induced_charge_e_histo")
    no_bins = electron.GetNbinsX()
    y_list = []
    for i in range(no_bins):
        y_list.append(electron.GetBinContent(i))
    y_max = max(np.abs(y_list))
    y_max_correct = ((y_max*sc.constants.e)/(0.01*(10**(-9))))*(10**6)
    
    return y_max_correct
    

#Code to firstly find the peaks of the fits for the pulses

def fit_convolve_data(x_data, y_data, time, hole_data):
    max_points_r = []
    time_points_r = []
    max_points_c = []
    time_points_c = []
    error = []
    
    for x_pos, y_pos, h in zip(x_data, y_data, hole_data):
        if x_pos == [] and y_pos == []:
            max_points_r.append(0)
            time_points_r.append(0)
            max_points_c.append(0)
            time_points_c.append(0)
            
        else:
            for x, y in zip(x_pos, y_pos):  

                dx = x[1]-x[0]
                x_pdf = np.arange(x[0], x[-1], dx)        
                y_pdf = sc.stats.norm.pdf(x_pdf, 0.1, 0.075) 
                y_convolve = sc.signal.fftconvolve(np.abs(y), y_pdf, mode='full')*dx
                x_convolve = np.linspace(0, x[-1], len(y_convolve))
                
                hole_cont = np.abs(((h*sc.constants.e)/(0.01*(10**(-9))))*(10**6))
                    
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= 0.35 and x_convolve[i-1] < 0.35:
                        hole_index_c = i
                        break
                    else:
                        continue
                    
                def gauss(x,a,x0,sigma):
                    return a*np.exp(-(x-x0)**2/(2*sigma**2))

                x1 = np.array(x)
                y1 = np.array(y)
                n = len(x1)                         
                mean = sum(x1*y1)/n                   
                sigma = sum(y1*(x1-mean)**2)/n
                popt, pcov = sc.optimize.curve_fit(gauss, x, y, p0=[1,mean,sigma])
                y_fit = gauss(x1, popt[0], popt[1], popt[2])
                
                error.append(np.sqrt(np.diag(pcov)[0]))
                print(pcov[0])
                    
                hole_cont_c = np.abs(y_convolve[hole_index_c])
    
                
                fig = plt.figure()
                ax = fig.add_subplot(1,1,1)
                #ax.plot(x_convolve, y_convolve, label='Convolution')
                ax.plot(x, y, label='Simulated Data')
                #ax.errorbar(np.abs(list(x))[:max_index-3], fit1, yerr=np.sqrt(np.diag(pcov1)[0]), color='purple')
                ax.plot(x, y_fit, label='Gaussian Fit for Simulated Data')
                ax.set_title('Example Current Pulse for 10ps Laser and PIN Diode E_Field')
                ax.set_ylabel('Current (\u03bcA)')
                ax.set_xlabel('Time (ns)')
                ax.legend()
                
                max_point_raw = 0.75*max(np.abs(y_fit))-hole_cont
                max_points_r.append(max_point_raw)
                
                max_point_con = max(y_convolve)
                index_max = np.where(y_convolve==max_point_con)
                max_points_c.append(max_point_con)
                
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= time and x_convolve[i-1] < time:
                        time_index_c = i
                        break
                    else:
                        continue
                    
                for i in range(len(x)):
                    if x[i] >= time and x[i-1] < time:
                        time_index_r = i
                        break
                    else:
                        continue
                    
                time_point_raw = np.abs(y_fit[time_index_r])-hole_cont
                time_points_r.append(time_point_raw)
                
                time_point_con = np.abs(y_convolve[time_index_c])-hole_cont_c
                time_points_c.append(time_point_con)
        
    return max_points_r, time_points_r, max_points_c, time_points_c, error

def getting_e_field(time_points_r, electron_data, no_charge, error, position, time, file):
    e_field_tot_r = []
    e_field_error = []
    
    v_m = 1.53*(10**9)*(300**(-0.87))
    E_c = 1.01*(300**1.55)
    B = 0.0257*(300**0.66)
    
    def mu_e_field(p, f):
        myFile = TFile.Open(f, "read")
        data = myFile.Get("ElectricFieldReader")
        detector = data.Get("detector1")
        efield = detector.Get("field1d_z")
        no_bins = efield.GetNbinsX()
        bin_size = 0.72/no_bins
        
        pos = p*10**(-3)
        placement = -0.36
        for bin in range(no_bins):  
            placement += bin_size
            if placement >= pos:
                field_strength = efield.GetBinContent(bin+1)
                break
            
        return field_strength
    
    def weighting_potential(f):
        myFile = TFile.Open(f, "read")
        data = myFile.Get("WeightingPotentialReader")
        detector = data.Get("detector1")
        wfield = detector.Get("potential1d")
        no_bins = wfield.GetNbinsX()
        bin_size = 0.72/no_bins
        w_tot = []
        grad = []
        point = []
        count1 = 0
        for i in range(no_bins):
            w_tot.append(wfield.GetBinContent(i+1))
            count1 +=1
            if wfield.GetBinContent(i) != wfield.GetBinContent(i-1):
                point.append(((count1*bin_size)-0.36)*10**(-3))
                grad.append(wfield.GetBinContent(i))
        
        calc_field = np.gradient(np.interp(np.linspace(-0.36*10**(-3), 0.36*10**(-3), 12), point, grad), (0.72*10**(-3))/12)
        
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
        ax.plot(np.linspace(-0.36*10**(-3), 0.36*10**(-3), 12), np.interp(np.linspace(-0.36*10**(-3), 0.36*10**(-3), 12), point, grad), (0.72*10**(-3))/12)
        
        pos = p*10**(-3)
        placement = -0.36
        for bin in range(12):  
            placement += ((0.72)/(12))
            if placement >= pos:
                wpos = calc_field[bin]
                break
            
        #plt.figure()
        #ax.plot(np.linspace(-0.36*10**(-3), 0.36*10**(-3), no_bins), calc_field)
        wf = wpos/((0.36-placement)*10**(-3))
        #print(wpos, placement, wf)
        return wpos
        
    for n, t, p in zip(no_charge, time_points_r, position):
        val = t*(10**(-4))/int(float(n[0])*1000)
        charge = 1.6*(10**(-19))
        weighting_field = weighting_potential(file)
        mu_si = (v_m/E_c)*(1/((1+((np.abs(mu_e_field(p, file))/E_c)**B))**(1/B)))
        e_field_r = val/(charge*weighting_field*mu_si)
        e_field_tot_r.append(e_field_r)
        print(t, weighting_field, e_field_r)
        
    for n, t, p in zip(no_charge, error, position):
        val = t*(10**(-4))/int(float(n[0])*1000)
        charge = 1.6*(10**(-19))
        weighting_field = weighting_potential(file)
        mu_si = (v_m/E_c)*(1/((1+((np.abs(mu_e_field(p, file))/E_c)**B))**(1/B)))
        e_field_e = val/(charge*weighting_field*mu_si)
        e_field_error.append(e_field_e)
    
    x_efield = np.linspace(position[0], position[-1], 10000)
    y_efield = []
    for i in x_efield:
        y_dat = mu_e_field(i, file)
        y_efield.append(y_dat)
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(x_efield, y_efield, 'black', label='TCAD Simulated Electric Field')
    
    #ax.errorbar(position, fit2, yerr=np.sqrt(np.diag(pcovr)[1]), label='Straight Line Fit for Electric Field Data', color='blue')
    ax.plot(position, e_field_tot_r, 'red', label='Electric Field Data')
    #ax.plot(position, e_field_electron, 'green', label='Electric Field from Electron Count')
    #ax.errorbar(position, e_field_tot_r, yerr=e_field_error, color='blue', label='E_Field from Simulated Current Pulses')
    ax.set_title(f'Electric Field against Position at Maximum Point of Gaussian Fit')
    ax.set_ylabel('Electric Field (V/cm)')
    ax.set_xlabel('Position (\u03bcm)')
    #ax.set_ylim(-2000, 11000)
    ax.legend()

 
files= ['root_files/tcad_efield1_-340.root', 'root_files/tcad_efield1_-330.root', 'root_files/tcad_efield1_-320.root', 'root_files/tcad_efield1_-310.root', 'root_files/tcad_efield1_-300.root', 'root_files/tcad_efield1_-290.root', 'root_files/tcad_efield1_-280.root', 'root_files/tcad_efield1_-270.root', 'root_files/tcad_efield1_-260.root', 'root_files/tcad_efield1_-250.root', 'root_files/tcad_efield1_-240.root', 'root_files/tcad_efield1_-230.root', 'root_files/tcad_efield1_-220.root', 'root_files/tcad_efield1_-210.root', 'root_files/tcad_efield1_-200.root', 'root_files/tcad_efield1_-175.root', 'root_files/tcad_efield1_-150.root']
x_values, y_values, pulse_names, hole_dat = [], [], [], []
for dat in files:
    pulse, x_dat, y_dat, hole_data, diff = collect_pulse_data(dat)
    pulse_names.append(pulse)
    x_values.append(x_dat)
    y_values.append(y_dat)
    hole_dat.append(hole_data)

electron_data = []
for e in files:
    electron_val = get_electrons(e)
    electron_data.append(electron_val)

max_points_r, time_points_r, max_points_c, time_points_c, error1 = fit_convolve_data(x_values, y_values, 2, hole_dat) 
getting_e_field(time_points_r, electron_data, pulse_names, error1, [340, 330, 320, 310, 300, 290, 280, 270, 260, 250, 240, 230, 220, 210, 200, 175, 150], 0.04, "root_files/tcad_efield1_330.root")

In [ ]:
#Firstly select the correct graphs (100e or above) and get the data for it (in the correct form for the next function)
#Must enter in the correct order- smallest to largest

from ROOT import TCanvas, TGraph, TFile, TH1F, TH2D
import numpy as np
import re
import scipy as sc
import matplotlib.pyplot as plt
import array as arr

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("Accumulated", graph.GetTitle())
            searching1 = re.search(" = ", graph.GetTitle())
            if searching != None and name[:9] == "charge_ev":
                a = graph.GetTitle()[searching1.end():searching1.end()+4]
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
            
    return pulsed_array, data_x, data_y

#Code to firstly find the peaks of the fits for the pulses

def extract_data(x_data, y_data, pulse_names):
    acc_charge = arr.array('f', [])
    
    for x_pos, y_pos, ppos in zip(x_data, y_data, pulse_names):
        charge_z = arr.array('f', [])
            
        for x, y, p in zip(x_pos, y_pos, ppos):  
            
            grad = np.gradient(np.array(y[0]), np.array(x[0]))
            c = 0
            for i in grad:
                c+=1
                if i>15 and c>66:
                    idx = np.where(grad==i)
                    break
            
            #fig = plt.figure()
            #ax = fig.add_subplot(1,1,1)
            #ax.plot(x, y, label='Simulated Data')
            #plt.axvline(x= np.array(x)[0][idx][0])
            #ax.errorbar(np.abs(list(x))[:max_index-3], fit1, yerr=np.sqrt(np.diag(pcov1)[0]), color='purple')
            #ax.plot(x, y_fit, label='Fit for Simulated Data')
            #ax.set_title('Example Current Pulse for 10ps Laser and PIN Diode E_Field')
            #ax.set_ylabel('Current (\u03bcA)')
            #ax.set_xlabel('Time (ns)')
            #ax.legend()
            
            charge_point = np.abs((np.array(y)[0][idx])/(float(p[0])*1000))
            charge_z.append(charge_point)
        
        acc_charge = np.concatenate((acc_charge, charge_z))
        
    acc_charge = acc_charge.reshape((len(x_data), len(x_data[0])))
    print(acc_charge)
        
    return acc_charge

def plotting_depletion(acc_charge):
    
    fig, ax = plt.subplots()
    im = ax.imshow(acc_charge, extent= [175, 300, -450, 450])
    ax.set_title('Depletion Depth of PIN Diode')

    fig.colorbar(im, ax=ax, label='Charge (C)')

    plt.show()

files = [['root_files/tcad_depletion_450_-350.root', 'root_files/tcad_depletion_450_-340.root', 'root_files/tcad_depletion_450_-330.root', 'root_files/tcad_depletion_450_-320.root', 'root_files/tcad_depletion_450_-310.root', 'root_files/tcad_depletion_450_-300.root', 'root_files/tcad_depletion_450_-290.root', 'root_files/tcad_depletion_450_-280.root', 'root_files/tcad_depletion_450_-270.root', 'root_files/tcad_depletion_450_-260.root', 'root_files/tcad_depletion_450_-250.root', 'root_files/tcad_depletion_450_-240.root', 'root_files/tcad_depletion_450_-230.root', 'root_files/tcad_depletion_450_-220.root', 'root_files/tcad_depletion_450_-210.root', 'root_files/tcad_depletion_450_-200.root', 'root_files/tcad_depletion_450_-175.root', 'root_files/tcad_depletion_450_-150.root'], ['root_files/tcad_depletion_400_-350.root', 'root_files/tcad_depletion_400_-340.root', 'root_files/tcad_depletion_400_-330.root', 'root_files/tcad_depletion_400_-320.root', 'root_files/tcad_depletion_400_-310.root', 'root_files/tcad_depletion_400_-300.root', 'root_files/tcad_depletion_400_-290.root', 'root_files/tcad_depletion_400_-280.root', 'root_files/tcad_depletion_400_-270.root', 'root_files/tcad_depletion_400_-260.root', 'root_files/tcad_depletion_400_-250.root', 'root_files/tcad_depletion_400_-240.root', 'root_files/tcad_depletion_400_-230.root', 'root_files/tcad_depletion_400_-220.root', 'root_files/tcad_depletion_400_-210.root', 'root_files/tcad_depletion_400_-200.root', 'root_files/tcad_depletion_400_-175.root', 'root_files/tcad_depletion_400_-150.root'], ['root_files/tcad_depletion_350_-350.root', 'root_files/tcad_depletion_350_-340.root', 'root_files/tcad_depletion_350_-330.root', 'root_files/tcad_depletion_350_-320.root', 'root_files/tcad_depletion_350_-310.root', 'root_files/tcad_depletion_350_-300.root', 'root_files/tcad_depletion_350_-290.root', 'root_files/tcad_depletion_350_-280.root', 'root_files/tcad_depletion_350_-270.root', 'root_files/tcad_depletion_350_-260.root', 'root_files/tcad_depletion_350_-250.root', 'root_files/tcad_depletion_350_-240.root', 'root_files/tcad_depletion_350_-230.root', 'root_files/tcad_depletion_350_-220.root', 'root_files/tcad_depletion_350_-210.root', 'root_files/tcad_depletion_350_-200.root', 'root_files/tcad_depletion_350_-175.root', 'root_files/tcad_depletion_350_-150.root'], ['root_files/tcad_depletion_300_-350.root', 'root_files/tcad_depletion_300_-340.root', 'root_files/tcad_depletion_300_-330.root', 'root_files/tcad_depletion_300_-320.root', 'root_files/tcad_depletion_300_-310.root', 'root_files/tcad_depletion_300_-300.root', 'root_files/tcad_depletion_300_-290.root', 'root_files/tcad_depletion_300_-280.root', 'root_files/tcad_depletion_300_-270.root', 'root_files/tcad_depletion_300_-260.root', 'root_files/tcad_depletion_300_-250.root', 'root_files/tcad_depletion_300_-240.root', 'root_files/tcad_depletion_300_-230.root', 'root_files/tcad_depletion_300_-220.root', 'root_files/tcad_depletion_300_-210.root', 'root_files/tcad_depletion_300_-200.root', 'root_files/tcad_depletion_300_-175.root', 'root_files/tcad_depletion_300_-150.root'], ['root_files/tcad_depletion_250_-350.root', 'root_files/tcad_depletion_250_-340.root', 'root_files/tcad_depletion_250_-330.root', 'root_files/tcad_depletion_250_-320.root', 'root_files/tcad_depletion_250_-310.root', 'root_files/tcad_depletion_250_-300.root', 'root_files/tcad_depletion_250_-290.root', 'root_files/tcad_depletion_250_-280.root', 'root_files/tcad_depletion_250_-270.root', 'root_files/tcad_depletion_250_-260.root', 'root_files/tcad_depletion_250_-250.root', 'root_files/tcad_depletion_250_-240.root', 'root_files/tcad_depletion_250_-230.root', 'root_files/tcad_depletion_250_-220.root', 'root_files/tcad_depletion_250_-210.root', 'root_files/tcad_depletion_250_-200.root', 'root_files/tcad_depletion_250_-175.root', 'root_files/tcad_depletion_250_-150.root'], ['root_files/tcad_depletion_200_-350.root', 'root_files/tcad_depletion_200_-340.root', 'root_files/tcad_depletion_200_-330.root', 'root_files/tcad_depletion_200_-320.root', 'root_files/tcad_depletion_200_-310.root', 'root_files/tcad_depletion_200_-300.root', 'root_files/tcad_depletion_200_-290.root', 'root_files/tcad_depletion_200_-280.root', 'root_files/tcad_depletion_200_-270.root', 'root_files/tcad_depletion_200_-260.root', 'root_files/tcad_depletion_200_-250.root', 'root_files/tcad_depletion_200_-240.root', 'root_files/tcad_depletion_200_-230.root', 'root_files/tcad_depletion_200_-220.root', 'root_files/tcad_depletion_200_-210.root', 'root_files/tcad_depletion_200_-200.root', 'root_files/tcad_depletion_200_-175.root', 'root_files/tcad_depletion_200_-150.root'], ['root_files/tcad_depletion_150_-350.root', 'root_files/tcad_depletion_150_-340.root', 'root_files/tcad_depletion_150_-330.root', 'root_files/tcad_depletion_150_-320.root', 'root_files/tcad_depletion_150_-310.root', 'root_files/tcad_depletion_150_-300.root', 'root_files/tcad_depletion_150_-290.root', 'root_files/tcad_depletion_150_-280.root', 'root_files/tcad_depletion_150_-270.root', 'root_files/tcad_depletion_150_-260.root', 'root_files/tcad_depletion_150_-250.root', 'root_files/tcad_depletion_150_-240.root', 'root_files/tcad_depletion_150_-230.root', 'root_files/tcad_depletion_150_-220.root', 'root_files/tcad_depletion_150_-210.root', 'root_files/tcad_depletion_150_-200.root', 'root_files/tcad_depletion_150_-175.root', 'root_files/tcad_depletion_150_-150.root'], ['root_files/tcad_depletion_100_-350.root', 'root_files/tcad_depletion_100_-340.root', 'root_files/tcad_depletion_100_-330.root', 'root_files/tcad_depletion_100_-320.root', 'root_files/tcad_depletion_100_-310.root', 'root_files/tcad_depletion_100_-300.root', 'root_files/tcad_depletion_100_-290.root', 'root_files/tcad_depletion_100_-280.root', 'root_files/tcad_depletion_100_-270.root', 'root_files/tcad_depletion_100_-260.root', 'root_files/tcad_depletion_100_-250.root', 'root_files/tcad_depletion_100_-240.root', 'root_files/tcad_depletion_100_-230.root', 'root_files/tcad_depletion_100_-220.root', 'root_files/tcad_depletion_100_-210.root', 'root_files/tcad_depletion_100_-200.root', 'root_files/tcad_depletion_100_-175.root', 'root_files/tcad_depletion_100_-150.root'], ['root_files/tcad_depletion_50_-350.root', 'root_files/tcad_depletion_50_-340.root', 'root_files/tcad_depletion_50_-330.root', 'root_files/tcad_depletion_50_-320.root', 'root_files/tcad_depletion_50_-310.root', 'root_files/tcad_depletion_50_-300.root', 'root_files/tcad_depletion_50_-290.root', 'root_files/tcad_depletion_50_-280.root', 'root_files/tcad_depletion_50_-270.root', 'root_files/tcad_depletion_50_-260.root', 'root_files/tcad_depletion_50_-250.root', 'root_files/tcad_depletion_50_-240.root', 'root_files/tcad_depletion_50_-230.root', 'root_files/tcad_depletion_50_-220.root', 'root_files/tcad_depletion_50_-210.root', 'root_files/tcad_depletion_50_-200.root', 'root_files/tcad_depletion_50_-175.root', 'root_files/tcad_depletion_50_-150.root'], ['root_files/tcad_depletion_0_-350.root', 'root_files/tcad_depletion_0_-340.root', 'root_files/tcad_depletion_0_-330.root', 'root_files/tcad_depletion_0_-320.root', 'root_files/tcad_depletion_0_-310.root', 'root_files/tcad_depletion_0_-300.root', 'root_files/tcad_depletion_0_-290.root', 'root_files/tcad_depletion_0_-280.root', 'root_files/tcad_depletion_0_-270.root', 'root_files/tcad_depletion_0_-260.root', 'root_files/tcad_depletion_0_-250.root', 'root_files/tcad_depletion_0_-240.root', 'root_files/tcad_depletion_0_-230.root', 'root_files/tcad_depletion_0_-220.root', 'root_files/tcad_depletion_0_-210.root', 'root_files/tcad_depletion_0_-200.root', 'root_files/tcad_depletion_0_-175.root', 'root_files/tcad_depletion_0_-150.root'], ['root_files/tcad_depletion_-50_-350.root', 'root_files/tcad_depletion_-50_-340.root', 'root_files/tcad_depletion_-50_-330.root', 'root_files/tcad_depletion_-50_-320.root', 'root_files/tcad_depletion_-50_-310.root', 'root_files/tcad_depletion_-50_-300.root', 'root_files/tcad_depletion_-50_-290.root', 'root_files/tcad_depletion_-50_-280.root', 'root_files/tcad_depletion_-50_-270.root', 'root_files/tcad_depletion_-50_-260.root', 'root_files/tcad_depletion_-50_-250.root', 'root_files/tcad_depletion_-50_-240.root', 'root_files/tcad_depletion_-50_-230.root', 'root_files/tcad_depletion_-50_-220.root', 'root_files/tcad_depletion_-50_-210.root', 'root_files/tcad_depletion_-50_-200.root', 'root_files/tcad_depletion_-50_-175.root', 'root_files/tcad_depletion_-50_-150.root'], ['root_files/tcad_depletion_-100_-350.root', 'root_files/tcad_depletion_-100_-340.root', 'root_files/tcad_depletion_-100_-330.root', 'root_files/tcad_depletion_-100_-320.root', 'root_files/tcad_depletion_-100_-310.root', 'root_files/tcad_depletion_-100_-300.root', 'root_files/tcad_depletion_-100_-290.root', 'root_files/tcad_depletion_-100_-280.root', 'root_files/tcad_depletion_-100_-270.root', 'root_files/tcad_depletion_-100_-260.root', 'root_files/tcad_depletion_-100_-250.root', 'root_files/tcad_depletion_-100_-240.root', 'root_files/tcad_depletion_-100_-230.root', 'root_files/tcad_depletion_-100_-220.root', 'root_files/tcad_depletion_-100_-210.root', 'root_files/tcad_depletion_-100_-200.root', 'root_files/tcad_depletion_-100_-175.root', 'root_files/tcad_depletion_-100_-150.root'], ['root_files/tcad_depletion_-150_-350.root', 'root_files/tcad_depletion_-150_-340.root', 'root_files/tcad_depletion_-150_-330.root', 'root_files/tcad_depletion_-150_-320.root', 'root_files/tcad_depletion_-150_-310.root', 'root_files/tcad_depletion_-150_-300.root', 'root_files/tcad_depletion_-150_-290.root', 'root_files/tcad_depletion_-150_-280.root', 'root_files/tcad_depletion_-150_-270.root', 'root_files/tcad_depletion_-150_-260.root', 'root_files/tcad_depletion_-150_-250.root', 'root_files/tcad_depletion_-150_-240.root', 'root_files/tcad_depletion_-150_-230.root', 'root_files/tcad_depletion_-150_-220.root', 'root_files/tcad_depletion_-150_-210.root', 'root_files/tcad_depletion_-150_-200.root', 'root_files/tcad_depletion_-150_-175.root', 'root_files/tcad_depletion_-150_-150.root'], ['root_files/tcad_depletion_-200_-350.root', 'root_files/tcad_depletion_-200_-340.root', 'root_files/tcad_depletion_-200_-330.root', 'root_files/tcad_depletion_-200_-320.root', 'root_files/tcad_depletion_-200_-310.root', 'root_files/tcad_depletion_-200_-300.root', 'root_files/tcad_depletion_-200_-290.root', 'root_files/tcad_depletion_-200_-280.root', 'root_files/tcad_depletion_-200_-270.root', 'root_files/tcad_depletion_-200_-260.root', 'root_files/tcad_depletion_-200_-250.root', 'root_files/tcad_depletion_-200_-240.root', 'root_files/tcad_depletion_-200_-230.root', 'root_files/tcad_depletion_-200_-220.root', 'root_files/tcad_depletion_-200_-210.root', 'root_files/tcad_depletion_-200_-200.root', 'root_files/tcad_depletion_-200_-175.root', 'root_files/tcad_depletion_-200_-150.root'], ['root_files/tcad_depletion_-250_-350.root', 'root_files/tcad_depletion_-250_-340.root', 'root_files/tcad_depletion_-250_-330.root', 'root_files/tcad_depletion_-250_-320.root', 'root_files/tcad_depletion_-250_-310.root', 'root_files/tcad_depletion_-250_-300.root', 'root_files/tcad_depletion_-250_-290.root', 'root_files/tcad_depletion_-250_-280.root', 'root_files/tcad_depletion_-250_-270.root', 'root_files/tcad_depletion_-250_-260.root', 'root_files/tcad_depletion_-250_-250.root', 'root_files/tcad_depletion_-250_-240.root', 'root_files/tcad_depletion_-250_-230.root', 'root_files/tcad_depletion_-250_-220.root', 'root_files/tcad_depletion_-250_-210.root', 'root_files/tcad_depletion_-250_-200.root', 'root_files/tcad_depletion_-250_-175.root', 'root_files/tcad_depletion_-250_-150.root'], ['root_files/tcad_depletion_-300_-350.root', 'root_files/tcad_depletion_-300_-340.root', 'root_files/tcad_depletion_-300_-330.root', 'root_files/tcad_depletion_-300_-320.root', 'root_files/tcad_depletion_-300_-310.root', 'root_files/tcad_depletion_-300_-300.root', 'root_files/tcad_depletion_-300_-290.root', 'root_files/tcad_depletion_-300_-280.root', 'root_files/tcad_depletion_-300_-270.root', 'root_files/tcad_depletion_-300_-260.root', 'root_files/tcad_depletion_-300_-250.root', 'root_files/tcad_depletion_-300_-240.root', 'root_files/tcad_depletion_-300_-230.root', 'root_files/tcad_depletion_-300_-220.root', 'root_files/tcad_depletion_-300_-210.root', 'root_files/tcad_depletion_-300_-200.root', 'root_files/tcad_depletion_-300_-175.root', 'root_files/tcad_depletion_-300_-150.root'], ['root_files/tcad_depletion_-350_-350.root', 'root_files/tcad_depletion_-350_-340.root', 'root_files/tcad_depletion_-350_-330.root', 'root_files/tcad_depletion_-350_-320.root', 'root_files/tcad_depletion_-350_-310.root', 'root_files/tcad_depletion_-350_-300.root', 'root_files/tcad_depletion_-350_-290.root', 'root_files/tcad_depletion_-350_-280.root', 'root_files/tcad_depletion_-350_-270.root', 'root_files/tcad_depletion_-350_-260.root', 'root_files/tcad_depletion_-350_-250.root', 'root_files/tcad_depletion_-350_-240.root', 'root_files/tcad_depletion_-350_-230.root', 'root_files/tcad_depletion_-350_-220.root', 'root_files/tcad_depletion_-350_-210.root', 'root_files/tcad_depletion_-350_-200.root', 'root_files/tcad_depletion_-350_-175.root', 'root_files/tcad_depletion_-350_-150.root'], ['root_files/tcad_depletion_-400_-350.root', 'root_files/tcad_depletion_-400_-340.root', 'root_files/tcad_depletion_-400_-330.root', 'root_files/tcad_depletion_-400_-320.root', 'root_files/tcad_depletion_-400_-310.root', 'root_files/tcad_depletion_-400_-300.root', 'root_files/tcad_depletion_-400_-290.root', 'root_files/tcad_depletion_-400_-280.root', 'root_files/tcad_depletion_-400_-270.root', 'root_files/tcad_depletion_-400_-260.root', 'root_files/tcad_depletion_-400_-250.root', 'root_files/tcad_depletion_-400_-240.root', 'root_files/tcad_depletion_-400_-230.root', 'root_files/tcad_depletion_-400_-220.root', 'root_files/tcad_depletion_-400_-210.root', 'root_files/tcad_depletion_-400_-200.root', 'root_files/tcad_depletion_-400_-175.root', 'root_files/tcad_depletion_-400_-150.root'], ['root_files/tcad_depletion_-450_-350.root', 'root_files/tcad_depletion_-450_-340.root', 'root_files/tcad_depletion_-450_-330.root', 'root_files/tcad_depletion_-450_-320.root', 'root_files/tcad_depletion_-450_-310.root', 'root_files/tcad_depletion_-450_-300.root', 'root_files/tcad_depletion_-450_-290.root', 'root_files/tcad_depletion_-450_-280.root', 'root_files/tcad_depletion_-450_-270.root', 'root_files/tcad_depletion_-450_-260.root', 'root_files/tcad_depletion_-450_-250.root', 'root_files/tcad_depletion_-450_-240.root', 'root_files/tcad_depletion_-450_-230.root', 'root_files/tcad_depletion_-450_-220.root', 'root_files/tcad_depletion_-450_-210.root', 'root_files/tcad_depletion_-450_-200.root', 'root_files/tcad_depletion_-450_-175.root', 'root_files/tcad_depletion_-450_-150.root']]
x_values, y_values, pulse_names = [], [], []
for dat in files:

    x, y, p = [], [], []
    for d in dat:
        pulse, x_dat, y_dat = collect_pulse_data(d)
        p.append(pulse)
        x.append(x_dat)
        y.append(y_dat)
        
    pulse_names.append(p)
    x_values.append(x)
    y_values.append(y)

acc_charge = extract_data(x_values, y_values, pulse_names) 
plotting_depletion(acc_charge)

In [ ]:
from ROOT import TCanvas, TGraph, TFile, TH1F, TH2D
import numpy as np
import re
import scipy as sc
import matplotlib.pyplot as plt

In [ ]:
#Firstly select the correct graphs (100e or above) and get the data for it (in the correct form for the next function)
#Must enter in the correct order- smallest to largest

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("e ", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                a = graph.GetTitle()[42:searching.start()]
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
                
    transient = myFile.Get("TransientPropagation")
    tdetector = transient.Get("detector1")
    hole = tdetector.Get("induced_charge_h_histo")
   # hy = hole.GetBinContent(7)
    #y_list = []
    #for i in range(2500):
        #y_list.append(hole.GetBinContent(i))
    #y_max = max(y_list)
    #diff = (y_max - hy)/2
    
    return pulsed_array, data_x, data_y#, hy, diff

#Code to firstly find the peaks of the fits for the pulses

def fit_convolve_data(x_data, y_data, time, sigma):
    time_points_r = []
    error1 = []
    error2 = []
    
    for x_pos, y_pos in zip(x_data, y_data):
        if x_pos == [] and y_pos == []:
            time_points_r.append(0)
            error.append(0)
            
        else:
            for x, y, s in zip(x_pos, y_pos, sigma):  
                
                def func1(x, B):
                    return 0*x +B 
                
                for i in range(len(y)):
                    if np.abs(y[i]) < 0.00015 and np.abs(y[i]) != 0 and i>5:
                        hole_index = i
                        break
                    else:
                        continue
                    
                for i in range(len(y)):
                    if np.abs(y[i]) > 0.00027 and i<4:
                        electron_index = i
                        break
                    else:
                        continue
                
                
                popt1, pcov1 = sc.optimize.curve_fit(func1, np.abs(list(x))[hole_index:66], list(y)[hole_index:66], sigma=s[hole_index:66])
                fit1 = list(func1(np.array(x)[hole_index:66], popt1[0]))
                
                popt2, pcov2 = sc.optimize.curve_fit(func1, np.abs(list(x))[electron_index:hole_index-3], list(y)[electron_index:hole_index-3], sigma=s[electron_index:hole_index-3])
                fit2 = list(func1(np.array(x)[electron_index:hole_index-3], popt2[0]))
                
                B_err1 = np.sqrt(pcov1[0, 0])
                error1.append(B_err1)
                B_err2 = np.sqrt(pcov2[0, 0])
                error2.append(B_err2)
                
                residuals1 = np.array(y)[hole_index:66] - np.array(fit1)
                residuals1 = np.where(residuals1 == 0, 1e-6, B_err1)
                chi_val1 = sum((residuals1**2 / np.array(B_err1)))
                
                residuals2 = np.array(y)[electron_index:hole_index-3] - np.array(fit2)
                residuals2 = np.where(residuals2 == 0, 1e-6, B_err2)
                chi_val2 = sum((residuals2**2 / np.array(B_err2)))
                    
                time_point_raw = ((min(fit2)+max(fit2))/2)-((max(fit1)+min(fit1))/2)
                time_points_r.append(time_point_raw)
                
                
                fig = plt.figure()
                ax = fig.add_subplot(1,1,1)
                #ax.plot(np.abs(list(x))[hole_index:-5], fit1)
                ax.errorbar(list(x)[hole_index:66], fit1, yerr=B_err1, color='green', label=f'Hole Top-Hat Straight Line Fit \nChi-Squared:{chi_val1:.5f}')
                #ax.plot(np.abs(list(x))[electron_index:hole_index-3], fit2)
                ax.errorbar(list(x)[electron_index:hole_index-3], fit2, yerr=B_err2, color='red', label=f'Electron Top-Hat Straight Line Fit\nChi-Squared:{chi_val2:.5f}')
                ax.plot(x,y, label='Simulated Data')
                #ax.set_title('Example Current Pulse for 10ps Laser and Overbiased Electric Field')
                ax.set_ylabel('Current (\u03bcA)', fontsize=13)
                ax.set_xlabel('Time (ns)', fontsize=13)
                plt.xticks(fontsize=12)
                plt.yticks(fontsize=12)
                ax.legend(fontsize=11)
                
                fig, axs = plt.subplots(2,1)
                fig.suptitle('Vertically stacked subplots')
                print(residuals1, fit1)
                axs[0].plot(np.array(residuals1), np.array(fit1))
                axs[1].plot(np.array(residuals2), np.array(fit2))
                axs[0].set_ylabel('Current (\u03bcA)', fontsize=13)
                axs[1].set_ylabel('Current (\u03bcA)', fontsize=13)
                axs[1].set_xlabel('Residuals', fontsize=13)
                plt.xticks(fontsize=12)
                plt.yticks(fontsize=12)
    
    return time_points_r, error1, error2

def getting_e_field(time_points_r, no_charge, error1, error2, position, time):
    e_field_tot_r = []
    e_field_tot_err1 = []
    e_field_tot_err2 = []
    
    v_m = 1.53*(10**9)*(300**(-0.87))
    E_c = 1.01*(300**1.55)
    B = 0.0257*(300**0.66)
        
    for n, t, p in zip(no_charge, time_points_r, position):
        print(t)
        val = t*(10**(-4))/np.abs(int(float(n[0])))
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((500000/E_c)**B))**(1/B)))
        e_field_r = val/(charge*weighting_field*mu_si)
        e_field_tot_r.append(e_field_r)
    
    for n, t, p in zip(no_charge, error1, position):
        val = t*(10**(-4))/np.abs(int(float(n[0])))
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((500000/E_c)**B))**(1/B)))
        e_field_e1 = val/(charge*weighting_field*mu_si)
        e_field_tot_err1.append(e_field_e1)
        
    for n, t, p in zip(no_charge, error2, position):
        val = t*(10**(-4))/np.abs(int(float(n[0])))
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((500000/E_c)**B))**(1/B)))
        e_field_e2 = val/(charge*weighting_field*mu_si)
        e_field_tot_err2.append(e_field_e2)

    total_error = np.sqrt((np.array(e_field_tot_err1))**2 + (np.array(e_field_tot_err2))**2)
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, [-500000, -500000, -500000,-500000, -500000, -500000,-500000, -500000], 'black', label='Allpix Simulated Electric Field')
     
    #ax.errorbar(position, fit1, yerr=np.sqrt(np.diag(pcov)[1]), label='Straight Line Fit for Convolution Data', color='purple')
    #ax.plot(position, e_field_tot_c, 'green', label=' Convolution Electric Field Data')
    ax.errorbar(position, e_field_tot_r, yerr=total_error, label='Electric Field From Simulated Current Pulses', color='blue')
    #ax.plot(position, e_field_tot_r, 'red', label='Electric Field Data')
    #ax.plot(np.linspace(5, 45, 5), [100000,100000,100000,100000,100000])
    #ax.set_title(f'Electric Field against Position for a Single Electron Pulse')
    ax.set_ylabel('Electric Field (V/cm)', fontsize=13)
    ax.set_xlabel('Position (\u03bcm)', fontsize=13)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    ax.set_ylim(-550000,-450000)
    ax.legend(fontsize=11)

 
files = ["root_files/one_e_n40.root", "root_files/one_e_n35.root", "root_files/one_e_n30.root", "root_files/one_e_n25.root", "root_files/one_e_n20.root", "root_files/one_e_n15.root", "root_files/one_e_n10.root", "root_files/one_e_n5.root",]
x_values, y_values, pulse_names = [], [], []
for dat in files:
    pulse, x_dat, y_dat = collect_pulse_data(dat)
    pulse_names.append(pulse)
    x_values.append(x_dat)
    y_values.append(y_dat)

time_points_r, error1, error2 = fit_convolve_data(x_values, y_values, 0.08, [[0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001]]) 
getting_e_field(time_points_r, pulse_names, error1, error2, [40, 35, 30, 25, 20, 15, 10, 5], 0.08)

In [ ]:
#Firstly select the correct graphs (100e or above) and get the data for it (in the correct form for the next function)
#Must enter in the correct order- smallest to largest

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("ke", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                a = graph.GetTitle()[42:searching.start()]
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
                
    transient = myFile.Get("TransientPropagation")
    tdetector = transient.Get("detector1")
    electron = tdetector.Get("induced_charge_e_histo")
    ey = np.abs(electron.GetBinContent(7))
    print(ey)
    y_list = []
    for i in range(2500):
        y_list.append(electron.GetBinContent(i))
    y_max = max(np.abs(y_list))
    diff = (y_max - ey)/2
    
    return pulsed_array, data_x, data_y, ey, diff

#Code to firstly find the peaks of the fits for the pulses

def fit_convolve_data(x_data, y_data, time, hole_data, sigma):
    max_points_r = []
    time_points_r = []
    max_points_c = []
    time_points_c = []
    error1 = []
    error2 = []
    
    for x_pos, y_pos, h in zip(x_data, y_data, hole_data):
        if x_pos == [] and y_pos == []:
            max_points_r.append(0)
            time_points_r.append(0)
            max_points_c.append(0)
            time_points_c.append(0)
            
        else:
            for x, y, s in zip(x_pos, y_pos, sigma):  

                dx = x[1]-x[0]
                x_pdf = np.arange(x[0], x[-1], dx)        
                y_pdf = sc.stats.norm.pdf(x_pdf, 0.1, 0.075) 
                y_convolve = sc.signal.fftconvolve(np.abs(y), y_pdf, mode='full')*dx
                x_convolve = np.linspace(0, x[-1], len(y_convolve))
                
                #hole_cont = ((h*sc.constants.e)/(0.01*(10**(-9))))*(10**6)
                    
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= 3 and x_convolve[i-1] < 3:
                        hole_index_c = i
                        break
                    else:
                        continue
                    
                for i in range(len(x)):
                    if x[i] >= 2 and x[i-1] < 2:
                        hole_index_r = i
                        break
                    else:
                        continue
                    
                for i in range(len(x)):
                    if x[i] >= 2.7 and x[i-1] < 2.7:
                        end_hole = i
                        break
                    else:
                        continue
                    
                
                def func1(x, B):
                    y = 0*x -B 
                    return y
                
                def func2(x, A, B):
                    return A*x-B
                                    
                for i in range(len(y)):
                    if np.abs(y[i]) < 0.4 and np.abs(y[i]) != 0 and i>5:
                        hole_index = i
                        break
                    else:
                        continue
                    
                for i in range(len(y)):
                    if np.abs(y[i]) > 0.45 and i<7:
                        electron_index = i
                        break
                    else:
                        continue
                
                popt1, pcov1 = sc.optimize.curve_fit(func1, list(x)[hole_index+10:end_hole], list(y)[hole_index+10:end_hole], sigma=s[hole_index+10:end_hole])
                fit1 = list(func1((np.array(x)[hole_index+10:end_hole]), popt1[0]))
                
                popt2, pcov2 = sc.optimize.curve_fit(func2, list(x)[electron_index:hole_index-3], list(y)[electron_index:hole_index-3], sigma=s[electron_index:hole_index-3])
                fit2 = list(func2(np.array(x)[electron_index:hole_index-3], popt2[0], popt2[1]))
                
                m_err1, c_err1 = np.sqrt(np.diag(pcov2))
                y_err1 = np.sqrt((m_err1 * np.array(x)[electron_index:hole_index-3])**2 + c_err1**2)
                print(y_err1, len(y_err1), len(x[electron_index:hole_index-3]))
                B_err = np.sqrt(pcov1[0, 0])
                
                
                error_list1 = []
                error_list2 = []
                chi_squared1 = []
                chi_squared2 = []
                
                residuals1 = np.array(y)[hole_index+10:end_hole] - np.array(fit1)
                residuals1 = np.where(residuals1 == 0, 1e-6, B_err)
                chi_val1 = sum((residuals1**2 / np.array(B_err)))
                
                residuals2 = np.array(y)[electron_index:hole_index-3] - np.array(fit2)
                residuals2 = np.where(residuals2 == 0, 1e-6, y_err1)
                chi_val2 = sum((residuals2**2 / np.array(B_err)))
                
                
                fig = plt.figure()
                ax = fig.add_subplot(1,1,1)
                #ax.plot(np.abs(list(x))[hole_index:-5], fit1)
                ax.errorbar(list(x)[hole_index+10:end_hole], fit1, yerr=B_err, color='green', label=f'Hole Top-Hat Straight Line Fit\nChi-Squared: {chi_val1:.2f}')
                #ax.plot(np.abs(list(x))[electron_index:hole_index-3], fit2)
                ax.errorbar(list(x)[electron_index:hole_index-3], fit2, yerr=y_err1, color='red', label=f'Electron Top-Hat Straight Line Fit\nChi-Squared: {chi_val2:.2f}')
                ax.plot(x, y, label='Simulated Data')
                #ax.set_title('Example Current Pulse for 10ps Laser and Overbiased Electric Field')
                plt.xticks(fontsize=12)
                plt.yticks(fontsize=12)
                ax.set_ylabel('Current (\u03bcA)', fontsize=13)
                ax.set_xlabel('Time (ns)', fontsize=13)
                ax.legend(fontsize=11)
                
                hole_cont_c = np.abs(y_convolve[hole_index_c])
                hole_cont = fit1[-1]
                
                max_point_raw = max(np.abs(y))-hole_cont
                max_points_r.append(max_point_raw)
                
                max_point_con = max(y_convolve)
                index_max = np.where(y_convolve==max_point_con)
                max_points_c.append(max_point_con)
                
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= time and x_convolve[i-1] < time:
                        time_index_c = i
                        break
                    else:
                        continue
                    
                for i in range(len(x)):
                    if x[i] >= time and x[i-1] < time:
                        time_index_r = i
                        break
                    else:
                        continue
                
                time_point_raw = fit2[time_index_r-1]-hole_cont
                time_points_r.append(time_point_raw)
            
                time_point_con = np.abs(y_convolve[time_index_c])-hole_cont_c
                time_points_c.append(time_point_con)
                
                y_erroring = y_err1[time_index_r]
                error1.append(y_erroring)
                
                error2.append(B_err)
        
    return max_points_r, time_points_r, max_points_c, time_points_c, error1, error2

def getting_e_field(time_points_r, time_points_c, no_charge, error, error2, position, time):
    e_field_tot_r = []
    e_field_tot_c = []
    e_field_tot_err = []
    e_field_tot_err2 = []
    
    v_m = 1.53*(10**9)*(300**(-0.87))
    E_c = 1.01*(300**1.55)
    B = 0.0257*(300**0.66)
        
    for n, t, p in zip(no_charge, time_points_c, position):
        val = t*(10**(-4))/np.abs(int(float(n[0])*1000))
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((5000/E_c)**B))**(1/B)))
        e_field_c = val/(charge*weighting_field*mu_si)
        e_field_tot_c.append(e_field_c)
        
    for n, t, p in zip(no_charge, time_points_r, position):
        val = t*(10**(-4))/np.abs(int(float(n[0])*1000))
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+(((5000)/E_c)**B))**(1/B)))
        e_field_r = val/(charge*weighting_field*mu_si)
        e_field_tot_r.append(e_field_r)
        
    for n, t, p in zip(no_charge, error, position):
        val = t*(10**(-4))/np.abs(int(float(n[0])*1000))
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((5000/E_c)**B))**(1/B)))
        e_field_e = val/(charge*weighting_field*mu_si)
        e_field_tot_err.append(e_field_e)
        
    for n, t, p in zip(no_charge, error2, position):
        val = t*(10**(-4))/np.abs(int(float(n[0])*1000))
        charge = 1.6*(10**(-19))
        weighting_field = 1/(100*(10**(-6)))
        mu_si = (v_m/E_c)*(1/((1+((5000/E_c)**B))**(1/B)))
        e_field_e2 = val/(charge*weighting_field*mu_si)
        e_field_tot_err2.append(e_field_e2)
    
    print(e_field_tot_err)
    total_error = np.sqrt((np.array(e_field_tot_err))**2 + (np.array(e_field_tot_err2))**2)
    print(total_error)
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(position, [-5000, -5000, -5000, -5000, -5000, -5000, -5000, -5000], 'black', label='Allpix Simulated Electric Field')
     
    #ax.errorbar(position, fit1, yerr=np.sqrt(np.diag(pcov)[1]), label='Straight Line Fit for Convolution Data', color='purple')
    #ax.plot(position, e_field_tot_c, 'green', label=' Convolution Electric Field Data')
    
    #ax.errorbar(position, fit2, yerr=np.sqrt(np.diag(pcovr)[1]), label='Straight Line Fit for Electric Field Data', color='blue')
    #ax.plot(position, e_field_tot_r, 'red', label='Electric Field Data')
    print(e_field_tot_r)
    #ax.plot(position[2:10], fit2, 'g--', label='Straight Line Fit for Real Data')
    ax.errorbar(position, e_field_tot_r, yerr=total_error, label='Electric Field From Simulated Current Pulses', color='blue')
    #ax.plot(position, e_field_tot_r, 'red', label='Electric Field Data')
    
    #ax.errorbar(position, e_field_tot_c, yerr=e_field_tot_err2, label='E_Field From Simulated Charge Pulses (electrons only)', color='orangered')
    #ax.plot(position, e_field_tot_c, 'green', label='Electric Field Data')
    #ax.set_title(f'Electric Field against Position of the Laser')
    ax.set_ylabel('Electric Field (V/cm)', fontsize=13)
    ax.set_xlabel('Position (\u03bcm)', fontsize=13)
    ax.set_ylim(-6500, -4000)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    ax.legend(fontsize=13)

 
files = ["root_files/ppd_n40.root", "root_files/ppd_n35.root", "root_files/ppd_n30.root", "root_files/ppd_n25.root","root_files/ppd_n20.root", "root_files/ppd_n15.root", "root_files/ppd_n10.root", "root_files/ppd_n05.root"]
x_values, y_values, pulse_names, e_data, error2 = [], [], [], [], []
for dat in files:
    pulse, x_dat, y_dat, electron_data, diff = collect_pulse_data(dat)
    pulse_names.append(pulse)
    x_values.append(x_dat)
    y_values.append(y_dat)
    e_dat = ((electron_data*sc.constants.e)/(0.01*(10**(-9))))*(10**6)
    e_data.append(e_dat)
    error2.append(((diff*sc.constants.e)/(0.01*(10**(-9))))*(10**6))
    
print(e_data)

max_points_r, time_points_r, max_points_c, time_points_c, error1, error12 = fit_convolve_data(x_values, y_values, 0.1, e_data, [[0.0001, 0.0003053994499903491, 0.005342808238100827, 0.021489582912657578, 0.030341356062923822, 0.020495028501924896, 0.0055821127162608295, 0.00511485041786709, 0.005272708395552107, 0.003600297197603962, 0.005171506822942218, 0.004486447378266849, 0.002462652100973566, 0.0034457732678365915, 0.005124897118380796, 0.004816014760495368, 0.003206949408822605, 0.004269995832685988, 0.00490906578632768, 0.006067056926240306, 0.007590835314100917, 0.007913622150334497, 0.00643693635603308, 0.009701383988122288, 0.009643314924857925, 0.008632192619892513, 0.009784747992487637, 0.00676303997724435, 0.005042157372694053, 0.0035179118943766225, 0.00274210692626758, 0.0017669401343129363, 0.0008858266563654818, 0.001896496167742248, 0.0009893440232258918, 0.001339049375842029, 0.0018673935900039439, 0.0014615752654570217, 0.0015953341958401992, 0.0013004079209536503, 0.0022238295134591694, 0.0023974109679944806, 0.001364140139214619, 0.002073116548251113, 0.0019448418371104147, 0.0020743275367749916, 0.002083724905874658, 0.0019136323519657848, 0.0023644711359950455, 0.0012243245161381016, 0.0018111352909483474, 0.001952258835962922, 0.0022695285166798827, 0.0010491788018728164, 0.0017043514423854943, 0.0006966366072865631, 0.0011635468193994445, 0.0022878593493732466, 0.0016199728983740088, 0.001173060968003595, 0.001544216768301846, 0.0016057895906514234, 0.0013774631391843966, 0.0014504562884454719, 0.0011095276877655619, 0.0022595226457166476, 0.0022633853540618776, 0.0013842024671452187, 0.0017182799950122118, 0.0018075619299932654, 0.0013555664645983308, 0.001882994494333742, 0.000953684550094275, 0.0016496415229720288, 0.0013856928313187977, 0.001981743190109913, 0.0014812603687686195, 0.0022020530847701226, 0.0016953332242326921, 0.0012032837604410685, 0.0011489435181880896, 0.0021101780467920016, 0.002702945967488853, 0.0017150586761889635, 0.001179744042389752, 0.0010208531699445803, 0.0019908719435876607, 0.0015342177943431015, 0.0018842657661238755, 0.0016100515768786138, 0.0013152188409291932, 0.001865575851807521, 0.0019736968004463585, 0.0012588418446774663, 0.0014346415909231754, 0.0013812411533890945, 0.001159178267945445, 0.0015149097284650323, 0.0017906135699819485, 0.0016502630003358684, 0.0018759327615523755, 0.0018387752765193712, 0.002237566274261161, 0.0015596239233566793, 0.0017660135776436613, 0.0023877642028867015, 0.0016028991462642985, 0.001590709375844631, 0.0013545515093671525, 0.0016224184573246623, 0.0010503802934376842, 0.0012905241799270784, 0.0016702581036613057, 0.0009578288201306201, 0.0018452809776890336, 0.0011616329783080152, 0.0016121482772761945, 0.0017952003260409707, 0.0013484581860768463, 0.0025652661759842112, 0.0016199485975076347, 0.0013939464487378304, 0.001630270548683732, 0.002042880242295129, 0.0019745560999645355, 0.0012863459612622917, 0.0017731916121306159, 0.0019309912140536886, 0.0013085670116423423, 0.001944968899639399, 0.0011540316898280539, 0.0016350583156459104, 0.002335521107388132, 0.0015643746872224934, 0.0016451301322869618, 0.0010900359118404532, 0.0017775633763889335, 0.001737613189589211, 0.0013390701367293953, 0.0014098174245537017, 0.0013915849409445807, 0.002080197669636082, 0.0014810328428360533, 0.0012012438438833049, 0.0013429169358529884, 0.0013954757936867736, 0.0019192144942851593, 0.0011255235539507733, 0.0020902078962272124, 0.0011455880112004482, 0.0016391204195127923, 0.0021010078441065427, 0.0020449185831553566, 0.0017690614490441436, 0.0016801740024583814, 0.0019246698143066886, 0.00171171654836514, 0.0012052463437604213, 0.001688327396563632, 0.001501242907507663, 0.00140573687779787, 0.0013622349096540356, 0.0012607744235583244, 0.002307977905216039, 0.0014213230343237703, 0.0013187290532856233, 0.0019692216096192498, 0.0017540667929816047, 0.002315274372135038, 0.0017918701944124697, 0.0014647129088717749, 0.00191956411699338, 0.0014284114168150593, 0.0015633760690120773, 0.0010694655376007989, 0.0021282117140656465, 0.001220032575641688, 0.0020746276223118022, 0.001953166493912876, 0.001165307187613857, 0.002006877638123785, 0.002009082894724007, 0.0014948869704252523, 0.001092513000518334, 0.0015821952530620108, 0.0015069908094401683, 0.002418048680629944, 0.0018992367509256289, 0.0015643438015317084, 0.0012531477854915956, 0.0011599755415801158, 0.0020922698330858355, 0.0020696778614399808, 0.0021244724124049245, 0.0017164353242365429, 0.0010414565611891395, 0.002345361359935322, 0.002361079726594359, 0.0015733322930436059, 0.0023098829422177024, 0.0017777249507389178, 0.0010968977832591772, 0.0011820875725909064, 0.0020496508938210494, 0.001580541964938565, 0.0010077042852044958, 0.0014064897774004259, 0.0025181152608187277, 0.0012546891753535032, 0.0018787026512670842, 0.0010335618377924113, 0.001983141019775117, 0.0019190709043967373, 0.0013571514338825588, 0.0019981420888761465, 0.0023234779580490143, 0.0017450010073669906, 0.0015132893692688194, 0.0011320461568850254, 0.0025092864461305204, 0.000980354971298139, 0.0015582211610852192, 0.0017797189657734124, 0.0011580969693594208, 0.0018242122386362902, 0.0018154333574866934, 0.0014126725416879233, 0.0018543427497429434, 0.0016648621302653701, 0.001649173046767534, 0.0020631251862423847, 0.0021654363281717647, 0.0016551937640461355, 0.002504954226838313, 0.0017852560678413292, 0.0010250688616748576, 0.001616663809398758, 0.0015499152527518384, 0.0020713702853254992, 0.0011204824855508856, 0.0017618686252527083, 0.0019000157584504883, 0.0016182547265657101, 0.0018176748990849734, 0.0018294651936207125, 0.0017271504014517286, 0.00140481238270322, 0.0022121699362965003, 0.0015644709315394151, 0.0012446905284203267, 0.0014805279646746708, 0.0016396818654692438, 0.002346839623274972, 0.0014576154022870025, 0.001515487026313108, 0.0015410865892219853, 0.0017983816395529797, 0.0012805224750580297, 0.0013487937648978816, 0.0013163279439327726, 0.0014566224722970145, 0.0014416550218801553, 0.0013860321098272364, 0.001290292068015825, 0.0012795639048646481, 0.0008985461823639676, 0.0022496335861470567, 0.002076482215563399, 0.0020924333122289726, 0.0018995772797281818, 0.0018531393805453148, 0.0014108281811626312, 0.0022429235398885414, 0.0015627224165222023, 0.0011314763990781173, 0.001590816516858682, 0.0015195931183864615, 0.0017890871680726838, 0.0010994267492253744, 0.0023512936715719217, 0.0011966690118270877, 0.0009025702559236908, 0.002009882251984547, 0.0016406340120475019, 0.0010252974957909288, 0.0016539829610426112, 0.0016283809486844318, 0.001306792756781789, 0.0018419446863811133, 0.0018919567572035984, 0.001337244324434165, 0.0016649804529638838, 0.002072413042842113, 0.0017350709368094271, 0.0011772513033336107, 0.0021662624654151647, 0.001693588229572256, 0.0016471972993955307, 0.0017059730877896355, 0.002301149728223657, 0.0008937233589618645, 0.0009020647182161851, 0.0018671551936010526, 0.001252513457302126, 0.0016743476074836253, 0.000821175092793055, 0.0013668112115044115, 0.0008129186668679806, 0.0009799700770859426, 0.0018896981600920906, 0.0016583235833012987, 0.0017288502666297939, 0.0013040067743060734, 0.001454502737298194, 0.0014761079689245798, 0.0018054821849066113, 0.0019337583160014338, 0.0012353345312559516, 0.0019827893672486542, 0.0017520319716153464, 0.0009170046352130932, 0.0019240549955473482, 0.0017544429030978558]]) 
print(time_points_r)
getting_e_field(time_points_r, e_data, pulse_names, error1, error12, [40, 35, 30, 25, 20, 15, 10, 5], 0.08)

In [ ]:
#Firstly select the correct graphs (100e or above) and get the data for it (in the correct form for the next function)
#Must enter in the correct order- smallest to largest

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("current", graph.GetTitle())
            searching1 = re.search(" = ", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                a = graph.GetTitle()[searching1.end():searching1.end()+4]
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
                
    transient = myFile.Get("TransientPropagation")
    tdetector = transient.Get("detector1")
    hole = tdetector.Get("induced_charge_h_histo")
    hy = hole.GetBinContent(7)
    y_list = []
    for i in range(2500):
        y_list.append(hole.GetBinContent(i))
    y_max = max(np.abs(y_list))
    diff = (y_max - hy)/2
    
    return pulsed_array, data_x, data_y, hy, diff

def get_electrons(file):
    myFile = TFile.Open(file, "read")
    transient = myFile.Get("TransientPropagation")
    tdetector = transient.Get("detector1")
    electron = tdetector.Get("induced_charge_e_histo")
    no_bins = electron.GetNbinsX()
    y_list = []
    for i in range(no_bins):
        y_list.append(electron.GetBinContent(i))
    y_max = max(np.abs(y_list))
    y_max_correct = ((y_max*sc.constants.e)/(0.01*(10**(-9))))*(10**6)
    
    return y_max_correct
    

#Code to firstly find the peaks of the fits for the pulses

def fit_convolve_data(x_data, y_data, time, hole_data, sig):
    max_points_r = []
    time_points_r = []
    max_points_c = []
    time_points_c = []
    error = []
    
    for x_pos, y_pos, h in zip(x_data, y_data, hole_data):
        if x_pos == [] and y_pos == []:
            max_points_r.append(0)
            time_points_r.append(0)
            max_points_c.append(0)
            time_points_c.append(0)
            
        else:
            for x, y, s in zip(x_pos, y_pos, sig):  

                dx = x[1]-x[0]
                x_pdf = np.arange(x[0], x[-1], dx)        
                y_pdf = sc.stats.norm.pdf(x_pdf, 0.1, 0.075) 
                y_convolve = sc.signal.fftconvolve(np.abs(y), y_pdf, mode='full')*dx
                x_convolve = np.linspace(0, x[-1], len(y_convolve))
                
                    
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= 0.35 and x_convolve[i-1] < 0.35:
                        hole_index_c = i
                        break
                    else:
                        continue
                    
                def gauss(x,a,x0,sigma):
                    return a*np.exp(-(x-x0)**2/(2*sigma**2))

                x1 = np.array(x)
                y1 = np.array(y)
                n = len(x1[:100])                         
                mean = sum(x1*y1)/n 
                sigma = sum(y1*(x1-mean)**2)/n
                popt, pcov = sc.optimize.curve_fit(gauss, x1, y1, sigma=s)
                y_fit = gauss(x1, popt[0], popt[1], popt[2])
                
                perr = np.sqrt(np.diag(pcov))
                
                y_err = np.sqrt(
                    (perr[0] * np.exp(-0.5 * ((x - popt[1]) / popt[2]) ** 2)) ** 2 + 
                    (popt[0] * (x - popt[1]) * np.exp(-0.5 * ((x - popt[1]) / popt[2]) ** 2) / (popt[2] ** 2) * perr[1]) ** 2 +
                    (popt[0] * np.exp(-0.5 * ((x - popt[1]) / popt[2]) ** 2) * (x - popt[1]) ** 2 / (popt[2] ** 3) * perr[2]) ** 2
                )
                y_err = np.where(y_err == 0, 1e-6, y_err)
                residuals = np.array(y) - np.array(y_fit)
                residuals = np.where(residuals == 0, 1e-10, y_err)
                chi_squared = sum((residuals**2 / np.array(y_err)))
                print(y_err)
                
    
                fig, ax = plt.subplots(1)
                #ax.plot(x_convolve, y_convolve, label='Convolution')
                ax.plot(x, y, label='Simulated Data')
                #ax.errorbar(np.abs(list(x))[:max_index-3], fit1, yerr=np.sqrt(np.diag(pcov1)[0]), color='purple')
                ax.plot(x, y_fit, label=f'Gaussian Fit for Simulated Data \nChi-Squared:{chi_squared:.2f}')
                #ax.set_title('Example Current Pulse for 10ps Laser and PIN Diode E_Field')
                ax.set_ylabel('Current (\u03bcA)', fontsize=13)
                ax.set_xlabel('Time (ns)', fontsize=13)
                ax.legend(fontsize=11)
                plt.xticks(fontsize=12)
                plt.yticks(fontsize=12)
                
                fig1, ax1 = plt.subplots(1)
                ax1.scatter(residuals,np.array(y_fit))
                ax1.set_xlabel('Residuals', fontsize=13)
                ax1.set_ylabel('Current (\u03bcA)', fontsize=13)
                plt.xticks(fontsize=12)
                plt.yticks(fontsize=12)
                
                max_point_raw = min(y_fit)
                index_max = np.where(y_fit==max_point_raw)[0][0]
                max_points_r.append(max_point_raw)
                
                max_point_con = max(y_convolve)
                max_points_c.append(max_point_con)
                
                for i in range(len(x_convolve)):
                    if x_convolve[i] >= time and x_convolve[i-1] < time:
                        time_index_c = i
                        break
                    else:
                        continue
                    
                for i in range(len(x)):
                    if x[i] >= time and x[i-1] < time:
                        time_index_r = i
                        break
                    else:
                        continue
                    
                time_point_raw = np.abs(y_fit[time_index_r])
                time_points_r.append(time_point_raw)
                
                y_erroring = y_err[time_index_r]
                error.append(y_erroring)
                
                time_point_con = np.abs(y_convolve[time_index_c])
                time_points_c.append(time_point_con)
                
    
                #error_list = []
                #for r, f in zip(list(y)[:index_max], y_fit#[:index_max]):
                #    y_err = (r-f)**2
                #    error_list.append(y_err)
                ##error_on_mean = np.sqrt(sum(error_list)/(len(error_list)-1))
                #print(error_on_mean)
                #error.append(np.abs(error_on_mean))
        
    return max_points_r, time_points_r, max_points_c, time_points_c, error

def getting_e_field(time_points_r, electron_data, no_charge, error, position, time, file):
    e_field_tot_r = []
    e_field_error = []
    
    v_m = 1.53*(10**9)*(300**(-0.87))
    E_c = 1.01*(300**1.55)
    B = 0.0257*(300**0.66)
    
    def mu_e_field(p, f):
        myFile = TFile.Open(f, "read")
        data = myFile.Get("ElectricFieldReader")
        detector = data.Get("detector1")
        efield = detector.Get("field1d_z")
        no_bins = efield.GetNbinsX()
        bin_size = 0.72/no_bins
        
        pos = p*10**(-3)
        placement = -0.36
        for bin in range(no_bins):  
            placement += bin_size
            if placement >= pos:
                field_strength = efield.GetBinContent(bin+1)
                break
        if p == 360:
            field_strength = efield.GetBinContent(0)
            
        return field_strength
    
    def weighting_potential(f):
        myFile = TFile.Open(f, "read")
        data = myFile.Get("WeightingPotentialReader")
        detector = data.Get("detector1")
        wfield = detector.Get("potential1d")
        no_bins = wfield.GetNbinsX()
        bin_size = 0.72/no_bins
        w_tot = []
        grad = []
        point = []
        count1 = 0
        for i in range(no_bins):
            w_tot.append(wfield.GetBinContent(i+1))
            count1 +=1
            if wfield.GetBinContent(i) != wfield.GetBinContent(i-1):
                point.append(((count1*bin_size)-0.36)*10**(-3))
                grad.append(wfield.GetBinContent(i))
        
        calc_field = np.gradient(np.interp(np.linspace(-0.36*10**(-3), 0.36*10**(-3), int(no_bins/4)), point, grad), (0.72*10**(-3))/(int(no_bins/4)))
        
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
        ax.set_xlabel('Position (\u03bcm)', fontsize=13)
        ax.set_ylabel('Weighting Potential (arb. units)', fontsize=13)
        ax.set_xlim(300,360)
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        ax.plot(1000000*np.linspace(-0.36*10**(-3), 0.36*10**(-3), int(no_bins/4)), np.interp(np.linspace(-0.36*10**(-3), 0.36*10**(-3), int(no_bins/4)), point, grad))
        
        pos = p*10**(-3)
        print(p)
        placement = -0.36
        for bin in range(int(no_bins/4)):  
            placement += ((0.72)/(int(no_bins/4)))
            if placement >= pos:
                wpos = calc_field[bin]
                break
        if p == 360:
            wpos = calc_field[-1]
        elif p ==300:
            wpos = calc_field[bin-5]
        #plt.figure()
        #ax.plot(np.linspace(-0.36*10**(-3), 0.36*10**(-3), no_bins), calc_field)
        wf = wpos/((0.36-placement)*10**(-3))
        #print(wpos, placement, wf)
        return wpos
        
    for n, t, p in zip(no_charge, time_points_r, position):
        val = t*(10**(-4))/int(float(n[0])*1000)
        charge = 1.6*(10**(-19))
        weighting_field = weighting_potential(file)
        mu_si = (v_m/E_c)*(1/((1+((np.abs(mu_e_field(p, file))/E_c)**B))**(1/B)))
        e_field_r = val/(charge*weighting_field*mu_si)
        e_field_tot_r.append(e_field_r)
        print(val, charge, weighting_field, mu_si)
        
    for n, t, p in zip(no_charge, error, position):
        val = t*(10**(-4))/int(float(n[0])*1000)
        charge = 1.6*(10**(-19))
        weighting_field = weighting_potential(file)
        mu_si = (v_m/E_c)*(1/((1+((np.abs(mu_e_field(p, file))/E_c)**B))**(1/B)))
        e_field_e = val/(charge*weighting_field*mu_si)
        e_field_error.append(np.abs(e_field_e))
        
        print(e_field_error)
    
    x_efield = np.linspace(position[0], position[-1], 10000)
    y_efield = []
    for i in x_efield:
        y_dat = mu_e_field(i, file)
        y_efield.append(y_dat)
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(x_efield, y_efield, 'black', label='TCAD Simulated Electric Field')
    
    #ax.errorbar(position, fit2, yerr=np.sqrt(np.diag(pcovr)[1]), label='Straight Line Fit for Electric Field Data', color='blue')
    #ax.plot(position, e_field_tot_r, 'red', label='Electric Field Data')
    #ax.plot(position, e_field_electron, 'green', label='Electric Field from Electron Count')
    ax.errorbar(position, e_field_tot_r, yerr=e_field_error, color='blue', label='E_Field from Simulated Current Pulses')
    #ax.set_title(f'Electric Field against Position at Maximum Point of Gaussian Fit')
    ax.set_ylabel('Electric Field (V/cm)', fontsize=13)
    ax.set_xlabel('Position (\u03bcm)', fontsize=13)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    ax.set_ylim(-150000, 60000)
    ax.legend(fontsize=11)

files = ['root_files/tcad_deplete_dope_0.0_-360.0.root', 'root_files/tcad_deplete_dope_0.0_-355.0.root', 'root_files/tcad_deplete_dope_0.0_-350.0.root', 'root_files/tcad_deplete_dope_0.0_-345.0.root', 'root_files/tcad_deplete_dope_0.0_-340.0.root', 'root_files/tcad_deplete_dope_0.0_-335.0.root', 'root_files/tcad_deplete_dope_0.0_-330.0.root', 'root_files/tcad_deplete_dope_0.0_-325.0.root', 'root_files/tcad_deplete_dope_0.0_-320.0.root', 'root_files/tcad_deplete_dope_0.0_-315.0.root', 'root_files/tcad_deplete_dope_0.0_-310.0.root', 'root_files/tcad_deplete_dope_0.0_-305.0.root']#, 'root_files/tcad_deplete_dope_0.0_-300.0.root']#|, 'root_files/tcad_deplete_dope_0.0_-295.0.root', 'root_files/tcad_deplete_dope_0.0_-290.0.root', 'root_files/tcad_deplete_dope_0.0_-285.0.root', 'root_files/tcad_deplete_dope_0.0_-280.0.root', 'root_files/tcad_deplete_dope_0.0_-275.0.root', 'root_files/tcad_deplete_dope_0.0_-270.0.root', 'root_files/tcad_deplete_dope_0.0_-265.0.root', 'root_files/tcad_deplete_dope_0.0_-260.0.root', 'root_files/tcad_deplete_dope_0.0_-255.0.root', 'root_files/tcad_deplete_dope_0.0_-250.0.root']
x_values, y_values, pulse_names, hole_dat = [], [], [], []
for dat in files:
    pulse, x_dat, y_dat, hole_data, diff = collect_pulse_data(dat)
    pulse_names.append(pulse)
    x_values.append(x_dat)
    y_values.append(y_dat)
    hole_dat.append(hole_data)

electron_data = []
for e in files:
    electron_val = get_electrons(e)
    electron_data.append(electron_val)

max_points_r, time_points_r, max_points_c, time_points_c, error1 = fit_convolve_data(x_values, y_values, 0.35, hole_dat, [[0.0001, 0.0016504044047445563, 0.0001, 0.004061120055107364, 0.0001, 0.004061120055107364, 0.0057682642279522, 0.0036491916263657474, 0.0056982054163709425, 0.00496364062103569, 0.0037009356135154605, 0.005797831185131617, 0.004402247767880842, 0.0071356768944435, 0.013794585171759273, 0.008605652234285407, 0.013479049080169768, 0.013193160702392877, 0.008518977953077786, 0.010081849545733343, 0.015647778193632916, 0.020580680508120057, 0.023512890420188587, 0.02321479416223336, 0.012671766577370226, 0.032327232491486, 0.026243949326255244, 0.030030267938212615, 0.04311478416483585, 0.01838026314614444, 0.03178952018347337, 0.03465178463072666, 0.026168565413745896, 0.038545669428394116, 0.03976143899487403, 0.059737232708251424, 0.037324995245465974, 0.03767627933090031, 0.05338508612258199, 0.06342951874245661, 0.04528349191897653, 0.04913118410087714, 0.047081073738293226, 0.0657593323568894, 0.09637707963753493, 0.049673333271857, 0.03788021582024734, 0.04510577751880812, 0.04875736985594347, 0.035890660203228036, 0.051856254916002556, 0.027839924918672238, 0.049218836049343406, 0.037432549752637885, 0.05945190973422094, 0.04900345006797039, 0.033059984371205484, 0.03093278324883851, 0.049122603632401944, 0.04104609390417217, 0.0583636368795021, 0.037916358400639075, 0.044372952252453655, 0.06403060320528343, 0.03153046745042047, 0.04412527924248481, 0.04140629297753606, 0.039593322685028916, 0.039627663717541235, 0.05313069197394373, 0.03953141579704737, 0.04378104234773389, 0.04587100530398179, 0.06576748459734869, 0.038431327889534864, 0.03717666584314317, 0.06991998797425628, 0.05224452588165671, 0.059399986993972385, 0.03075484738036906, 0.036782186401233444, 0.044636255815977265, 0.026075727844150836, 0.054617615792542845, 0.03766550085505185, 0.03208735305938681, 0.03478785079726309, 0.037432486033920555, 0.040215660044515864, 0.033901188402926184, 0.037742058200120575, 0.05086879748887998, 0.037797383813993134, 0.039853291459925236, 0.03554517133501663, 0.03710405408332925, 0.045968413808773714, 0.03341426625835902, 0.045797581513458516, 0.04516935073840492, 0.032194603485798425, 0.025124141992106205, 0.036977054045906864, 0.03451555706634839, 0.02973602804235393, 0.027264325279799675, 0.02681902620374054, 0.03999653441418799, 0.02875837331491917, 0.04483875133443708, 0.040853943118229805, 0.036666626395065585, 0.026632369202316982, 0.041964859014205734, 0.02990758990996073, 0.038731036078063295, 0.03139836175356434, 0.02962107648170315, 0.03905869282217455, 0.030082133232119786, 0.016092161859046824, 0.031151804453241037, 0.021464126183534073, 0.031060652201473443, 0.027073271875257265, 0.02141166851122704, 0.02501508729828554, 0.02589491357201567, 0.026392850979712446, 0.021719020804236114, 0.016745839950740804, 0.01791200447217598, 0.021341473597584797, 0.021018867410618636, 0.026455043033367556, 0.01837282580821744, 0.031218629741142968, 0.024879419683550234, 0.021196930202732098, 0.01982545951501409, 0.016791033068293517, 0.014441704287731965, 0.01737565908260899, 0.014485699378647637, 0.014160340833743719, 0.012457564467555641, 0.012197789721877012, 0.015535438510542536, 0.01431038760097214, 0.010420456145035178, 0.011239200768990269, 0.01388894928040264, 0.010294866561537726, 0.011300682804377356, 0.01631391318322489, 0.00797292459753244, 0.013319530562268062, 0.017399351124533568, 0.015251621586894271, 0.009316807269762703, 0.012317950032934897, 0.014600136228779929, 0.016819055471458764, 0.007848921619760234, 0.02073094773135978, 0.018014050110786075, 0.011021046287096893, 0.019108919583330997, 0.01628569934307603, 0.01513077213013618, 0.010970502591650973, 0.012275739551499482, 0.008434443455309137, 0.008004481909949077, 0.01089558172378114, 0.008517199802461725, 0.015298188315446968, 0.01449976818978759, 0.009941397971820568, 0.008692556819348413, 0.014236529209422196, 0.014519954519052603, 0.015097674969519657, 0.017096449347182192, 0.016007136703252267, 0.012273334908964217, 0.014988862320950107, 0.013357196461810729, 0.01647199588378183, 0.011395061579327114, 0.011419629139826895, 0.006773410832096499, 0.012834524814721493, 0.016584262379150255, 0.011953569049875448, 0.010763797085733228, 0.010896431816198609, 0.008951624079928416, 0.007020781527141595, 0.008300458804263024, 0.012986946086940676, 0.012257692132631148, 0.00690729801285224, 0.01792154483535148, 0.005429477121108637, 0.0083808166447283, 0.008302719492943984, 0.007374246004753096, 0.018441256385101096, 0.012263348759729262, 0.02852284446824553, 0.02327792179192553, 0.017583935847177996, 0.01025985333885265, 0.017909723689349035, 0.023967484414140646, 0.013791193511888362, 0.013076337865902824, 0.01853942086443775, 0.021468889978486772, 0.014237196831717173, 0.019562932055801605, 0.014982397359309925, 0.016652416384725824, 0.021304078536985138, 0.01765858297937982, 0.016666466090249202, 0.020653015884001882, 0.017986076072045377, 0.012954135974855964, 0.01682189678908508, 0.016240026135812678, 0.009710699796714108, 0.008605128700216399, 0.01880724607359417, 0.0089510250716323, 0.010300026357333977, 0.007527677558254498, 0.009743524552647799, 0.012707095524003655, 0.01121248637731326, 0.010774256526856736, 0.009615950238974666, 0.013900992898957227, 0.011698161087923615, 0.011295847453840567, 0.008692369640303073, 0.009947057458204192, 0.009854053270849752, 0.01443396827393343, 0.010087917283682395, 0.010958971962537917, 0.006997290042445315, 0.009020436900192016, 0.0072228559180693005, 0.011919869486220794, 0.012223102448791046, 0.009019106016729396, 0.008400520176672737, 0.01755949131800562, 0.020498324595327584, 0.013267533493108332, 0.011398938458030528, 0.012275225065368992, 0.01158276770412997, 0.011517630695386593, 0.010146951690020861, 0.010865674919781089, 0.012808121459837956, 0.009789608304588913, 0.007001808339437227, 0.009395015519172796, 0.014355671485770808, 0.016537916894159392, 0.014280825660966808, 0.010433282815967311, 0.01058181751084855, 0.01732753563708114, 0.012032708384788692, 0.010498390096989538, 0.016308127784339165, 0.01239433243916545, 0.01150444482619535, 0.00508961385590277, 0.015366568857039239, 0.013039537484385308, 0.015518209779657427, 0.011733322072761116, 0.009150467443782175, 0.011527031523755584, 0.008037099564504269, 0.009326945023892863, 0.012521344541594412, 0.008749172162859427, 0.008097859691247324, 0.010388999026664247, 0.011147772985228007, 0.010251046456759515, 0.013809648043385796, 0.007541943596750158, 0.009257210214780577, 0.012840384478232297, 0.01137814608377671, 0.012713132681332171, 0.01709814982569831, 0.009618732342863465, 0.008525010948692712, 0.010415121258496286, 0.009568200773003573, 0.010590140461665358, 0.007300585646715693, 0.00872149894209669, 0.009814301196580751, 0.010451308108492034, 0.011721963123467082, 0.010871396887837703, 0.009544778953671675, 0.0064363579245455315, 0.008645780457299359, 0.006976775325635147, 0.00814692119195283, 0.0060652595354883, 0.006460306651840676, 0.00825205028042218, 0.005199005314784298, 0.004360588926074585, 0.009748200776471132, 0.0087369349797645, 0.01515010827846553, 0.010136365669890677, 0.009572384899714067, 0.009675041241902264, 0.00351724582729259, 0.008831708084248237, 0.006987826211898194, 0.00687152051612887, 0.008921090804900968, 0.013333831231996744, 0.011126633006773122, 0.005464276238072923, 0.006721388154384612, 0.006826401917496336, 0.007934394641367692, 0.013312386946344403, 0.009178479630843474, 0.007360628705772754, 0.009495191419158866, 0.005455961648103765, 0.012409413217429081, 0.009814696984779086, 0.008539515909978259, 0.006203926722377636, 0.008184169111317897, 0.011415316248193991, 0.0038040023638845357, 0.007599878494256155, 0.007380466352256459, 0.007907988146612479, 0.00804136299189503, 0.0040843724783520925, 0.011348307981546868, 0.007208337744652339, 0.00526609148285737, 0.00949655292502759, 0.0029503403390911663, 0.007433329930900542, 0.005553288634672294, 0.008340713488032525, 0.0052519265173231735, 0.006211737453168368, 0.007164724915812349, 0.006693148117632213, 0.009954455906614878, 0.008682630175123125, 0.00927483760076507, 0.009684611443941107, 0.006306367635683257, 0.006611747371974189, 0.007444415362942475, 0.003894679557252901, 0.003979713947468188, 0.005842299521093912, 0.004659243343126863, 0.005261671395791877, 0.000829550547663711, 0.007707987002938397, 0.00423839334482158, 0.005305719686213947, 0.001983784310511845, 0.0046013450446585336, 0.005558636352110705, 0.004917956546136827, 0.004950193236635532, 0.005532110290997005, 0.006526781346014403, 0.0047909725571187, 0.0043797267864728595, 0.005846925660993537, 0.002649878728823952, 0.006576111300562113, 0.005816039245534669, 0.0061904846013614775, 0.009375456701833202, 0.005025475133946879, 0.0033466097174620945, 0.002600595375880613, 0.006882932727895139, 0.0060461959160130275, 0.006156147443270637, 0.004695328325043347, 0.002704143698225467, 0.004118142639428278, 0.006466832642465497, 0.005944076243188905, 0.0063046095491621984, 0.007578509783870955, 0.006078274253535822, 0.0047140868504374645, 0.006382173113062654, 0.006121512746588499, 0.010897511187382272, 0.0031298628418927952, 0.009617899423660559, 0.009549748380772298, 0.006483400329347822, 0.005373201216436937, 0.006254638319240369, 0.007502511685315195, 0.007401515725680938, 0.005703550713462381, 0.004198502298981058, 0.002775449822012706, 0.002713489115284347, 0.002633327557165178, 0.004933058432301694, 0.0026776135264483334, 0.002880968450468406, 0.004799349386904369, 0.004594415974392453, 0.0020537920529073287, 0.004821452919915238, 0.0030291402752115615, 0.002024477203665616, 0.003012381483407262, 0.004279484404033868, 0.005744635787500979, 0.0026730659859359562, 0.0030399608270517153, 0.0047118365721058215, 0.0034399562080230247, 0.005088187126915835, 0.004494471771195865, 0.003121112587439428, 0.002878653247584875, 0.00563158606982368, 0.00643401875062982, 0.005425938960577648, 0.004811982870682829, 0.006370079025306324, 0.006310993202672726, 0.006537867890661625, 0.00425059187843976, 0.007454606596905641, 0.0030249326606685604, 0.004827689885159754, 0.0034111549962530527, 0.005513006886078339, 0.0019915189005573256, 0.002218582970561809, 0.0040836901055718005, 0.00622992840255045, 0.003480413985383089, 0.004465122122482982, 0.0020125290682680807, 0.003573530384031074, 0.00263179146117946, 0.003156349798781893, 0.004351530042580791, 0.00464987258870857, 0.004294794429965293, 0.004504844068455811, 0.004826732501730257, 0.006781141040846371, 0.006801125429729994, 0.004333899351827349, 0.0035804711045436546, 0.004581548500563788, 0.0039412459557535496, 0.008275129527118601, 0.009821370951921145, 0.008502878302833385, 0.0033500316685796906, 0.006576364917765024, 0.004663237355700345, 0.005906059530724897, 0.008259147242544902, 0.007111125996089549, 0.006666894717769202, 0.007039602267855085, 0.0043944348197259, 0.006581553515904184, 0.008243841862745255, 0.004236431221840372, 0.0029464801727963454, 0.006622822057501846, 0.00199083240948867, 0.004181380745125344, 0.004795699147045886, 0.004032573896162534, 0.004339314603851552, 0.005296328689438955, 0.004865168095325347, 0.0074063518244935475, 0.005419917223012319, 0.0020393537782418315, 0.002162685537397588, 0.0060263665015093375, 0.005010558620772349, 0.0069521732604213675, 0.005730207076645767, 0.006510422298429155, 0.004359343017853575, 0.008424826765562037, 0.006172565318132367, 0.003398687204967612, 0.004960986787975799, 0.007882169158310668, 0.007997191248061846, 0.004611922943357837, 0.0043217025751443065, 0.006092117512454755, 0.00591606937581606, 0.005022231796397429, 0.005251831355801099, 0.0030914786009950714, 0.004336678722207497, 0.003008870295929781, 0.004798445408671247, 0.0039624298320358456, 0.006004744218125429, 0.004064144499153025, 0.003884497947611204, 0.005308028011748954, 0.004143742876968577, 0.003220637610814892, 0.0008788539041745712, 0.004600973767632642, 0.007294188188908561, 0.0043434555755937805, 0.006047291676041121, 0.007210734994495618, 0.005572883912371037, 0.0037338034115885444, 0.006370410979683342, 0.004596428275004298, 0.005799001489029983, 0.007670136614082046, 0.004032035538200619, 0.003038428970595663, 0.004248180311978409, 0.0047613973601023825, 0.0030319102438329678, 0.004949181690708626, 0.0034558371316092615, 0.0006480062820490856, 0.004407286216638591, 0.003157846790292941, 0.0030748356563463394, 0.005244707075925611, 0.0025997374227486178, 0.006559732932235329, 0.006355770675902845, 0.004864632230363727, 0.006940022378737046, 0.005035325796100503, 0.006429560211874, 0.008096536527161377, 0.00819984530768689, 0.006190878788672, 0.005637005819683177, 0.002023557914321999, 0.004441140936227804, 0.002044154410033316, 0.0020811254403263375, 0.0034726113057014464, 0.001973363644748917, 0.0020440323714261293, 0.005153143549827602, 0.0040654273964629934, 0.003126715072059752, 0.0020352208427986862, 0.0014283795440898136, 0.005236287186594019, 0.007267429288457212, 0.004768609968048403, 0.006107162648340519, 0.005836478555529436, 0.0063922970985199066, 0.00422209225989101, 0.0006792467980810917, 0.002041884393161029, 0.004136537804572856, 0.0026228920301538104, 0.003560961714944042, 0.004056687249825463, 0.002087706485795049, 0.0047465741815995175, 0.004896259393954681, 0.00014855725623065218, 0.0020127355862761816, 0.003491950795806744, 0.0026738250393127066, 0.0022696700359611443, 0.0026249369720450656, 0.004283751514365026, 0.006312395926994915, 0.004315982502190063, 0.004910693170884127, 0.006133720012217516, 0.002583308797819962, 0.0068226453458839365, 0.007889739331353628, 0.0020038441944385497, 0.0036339604277552335, 0.004332830134297503, 0.007066022974504031, 0.002121749749489725, 0.003873907306169813, 0.006216449448495826, 0.005245915830407815, 0.0039995751143527885, 0.004386876157388087, 0.004284548148339156, 0.003484407693488387, 0.004329290915072021, 0.0036253906275043025, 0.001992500140751947, 0.003457631600926554, 0.0027283900739491886, 0.001997109070619949, 0.004080670459595722, 0.004748757557926529, 0.00404416748008976, 0.005060822147293632, 0.002610305456816146, 0.006361970887313452, 0.005260191844502256, 0.001977823858267268, 0.004729247347711595, 0.0042334499249202075, 0.0021024378091203558, 0.004403535674389501, 0.002633873228024217, 0.002699166616179328, 0.002078275027607552, 0.00204267832628885, 0.0009184208643989198, 0.0026921620706815415, 0.002251131990949424, 0.0001, 0.0025000394911768873, 0.004061090038982562, 0.004298520237243321, 0.00532273970556565, 0.00231047105072234, 0.002256544357815438, 0.002010321214133956, 0.0008749402344920227, 0.0030828504435039946, 0.0052002184818499605, 0.004693775275119705, 0.0042697578028884015, 0.0029149846756528655, 0.003082071094792248, 0.004693572192285353, 0.005587550761396351, 0.004647572445062834, 0.0044295969522686746, 0.004709117927162196, 0.0053273478124682425, 0.00369718972703603, 0.008037130195050113, 0.005483001045776659, 0.005017754218298912, 0.007770557742695036, 0.00953217006447647, 0.0066408471768061444, 0.0055291514040059, 0.004033520009827518, 0.006398769837987114, 0.007606172219714797, 0.005288006474935815, 0.002672916359661179, 0.005255352695966912, 0.00420442072564172, 0.0026744652123926664, 0.00430660935324501, 0.002017677882744968, 0.003149043852710325, 0.0032972971322766213, 0.00039150617038824937, 0.005961996499755769, 0.005803345200094889, 0.0006443651970471455, 0.001126775809421757, 0.0026678297547166117, 0.0048785067768255036, 0.008045951893285873, 0.006330997393636295, 0.004757286625537862, 0.010056349805697523, 0.004483565723541998, 0.008795208995134945, 0.006773886768960024, 0.002639951692432969, 0.006802634740797192, 0.007256106697576149, 0.00460834087278036, 0.004225151281110632, 0.0028604394747805584, 0.004069956906792221, 0.005958486913152084, 0.004210411492588083, 0.006375506623698046, 0.0051339904936614274, 0.006065081890253408, 0.005729055318136527, 0.002002346620532614, 0.0029534918833620167, 0.0020879584555417753, 0.004529072422344029, 0.002182051717489655, 0.0020331923466333406, 0.003146980306565429, 0.0037907065951353182, 0.005946897735979044, 0.0043338751236785345, 0.004915679956371483, 0.0027794362108269632, 0.007297970295425062, 0.006116937598670129, 0.00738248589875059, 0.0020296797844374876, 0.004882883445877829, 0.004567329574433348, 0.0033408711292204304, 0.004761729800752204, 0.006283599568613523, 0.0064042389093456374, 0.0036491727105051894, 0.005296669694572421, 0.006354181560738207, 0.008123987665447838, 0.007220204091863707, 0.005609270771616948, 0.004287927021479281, 0.005589369977874448, 0.0018164429968312546, 0.004630417811428551, 0.006605287642914569, 0.0006632831933070538, 0.00456226501196436, 0.0021349506875385353, 0.004688490914093619, 0.004900585896294021, 0.004711394940226789, 0.0046758121081696665, 0.004067455924406845, 0.003249387843011531, 0.004075273773332294, 0.003403395928672302, 0.0048108551901871275, 0.004067899580251696, 0.004490362356856376, 0.004432508215605024, 0.007055624685443807, 0.006735715130188741, 0.006095112001516842, 0.004202808677118256, 0.0020189366336039742, 0.005464675837880077, 0.0020580133985160535, 0.004925986268117789, 0.004451837423684756, 0.0056163327810725295, 0.004437536959053432, 0.0067998831222259625, 0.006317854037130617, 0.0051104440261673404, 0.0030734594558929646, 0.0034507128522615134, 0.004788178183723685, 0.004622768631465873, 0.006386411947440284, 0.0053487391885568515, 0.004821958699372085, 0.006019259045716132, 0.004386244019791201, 0.005239007747272123, 0.004377246268447343, 0.002652847070082633, 0.003819575272953847, 0.003370473312464063, 0.002621290660086029, 0.0009381144134829596, 0.007682610342561517, 0.004144893122628947, 0.005247102400464635, 0.006588316572302305, 0.00372993868034522, 0.005346238710739426, 0.005264888207840011, 0.004368977384911379, 0.004481844534654025, 0.00693704237349297, 0.0006849952554492982, 0.0031892064832927352, 0.003673163476723762, 0.00461677813030056, 0.002316093967989035, 0.0039972162976130805, 0.003395142308317581, 0.002616417934600518, 0.0033282765160942977, 0.004153573991418338, 0.00403665524930921, 0.0027630933377554086, 0.002684727141424787, 0.0022728541637099158, 0.006965759474431036, 0.0048737447557005275, 0.00538945135054457, 0.005811620282665267, 0.004387575344329507, 0.005592747541388652, 0.005214383934460904, 0.006143348823847836, 0.00330147891507863, 0.005313478918595369, 0.008139735923247701, 0.008637209194673237, 0.009908982247104087, 0.00413609189508998, 0.005588286549163736, 0.007266805597221251, 0.005514802862916529, 0.002113820155532545, 0.00010445059004259197, 0.0031443001596755467, 0.0027249357565773532, 0.0028404100248406545, 0.005608758531300178, 0.006079276918060537, 0.006172703511738022, 0.005119984795727058, 0.0050023745843732065, 0.003076660040277041, 0.004032033889732846, 0.0006499523290163, 0.00644460425306035, 0.004456690452888645, 0.0019425860703063895, 0.0020395448243395067, 0.003617230605024069, 0.001965986155016828, 0.0021749692683355163, 0.004274722017529213, 0.004062218921012533, 0.004082698924669855, 0.007075459293944873, 0.005261832327006028, 0.0060608835736758486, 0.0036225959072463195, 0.009538568546278551, 0.0053714099935132644, 0.003803553157184017, 0.002545089707079992, 0.006379556794420237, 0.008479192227135968, 0.005356200785219303, 0.005061598300529001, 0.008568822819849862, 0.006951374091622552, 0.005115952654543953, 0.003310640585941066, 0.0006436126828753547, 0.0003038438630059455, 0.004655202027837284, 0.0011579728049746962, 0.00270584443481217, 0.00400075901506696, 0.0026876959699161916, 0.0006400925874945468, 0.0028136120079490924, 0.0041753012344186765, 0.005428875458821506, 0.0030417734726977106, 0.002658661547278577, 0.0008369598177259667, 0.0030960268793658874, 0.0020677742996693923, 0.0026207919208821114, 0.004075015595892438, 0.004740796097789925, 0.0021079077397889103, 0.0052844561269243025, 0.006052873740265818, 0.002008638480224656, 0.004182946655838249, 0.004328574574171767, 0.002592187611439265, 0.0006224890035666205, 0.004276965137442216, 0.00298596208353198, 0.0027895917082054263, 0.0050210731242984015, 0.0020078074163097843, 0.004252176003901166, 0.0020888381127990765, 0.006503622497809032, 0.007688864420495509, 0.0026839039102704893, 0.004522126740381464, 0.004088518412468566, 0.0049022045140371074, 0.006075988860001488, 0.006373617266791503, 0.004275842283871961, 0.0001, 0.0026039043764140833, 0.004746040431278742, 0.004291340887199848, 0.0010303593290128216, 0.007107998895599909, 0.00265286585598513, 0.0030953949421161854, 0.0021801910933685205, 0.0026862754238332356, 0.004061246216904523, 0.002869143789004452, 0.004000562605498432, 0.005235311561521042, 0.0024059757683922976, 0.004803113952714083, 0.004695955423557889, 0.0020122117079354692, 0.00622432418477308, 0.002293688499057159, 0.0020693284717340386, 0.0031741245948643223, 0.004525132510517334, 0.005632802867243741, 0.005360855472652133, 0.0059906672919747585, 0.0030874063747770098, 0.0020273132211746894, 0.004919060505086255, 0.008119657856805775, 0.008999205058090457, 0.005365945788899714, 0.0026503934828631492, 0.004060666555495558, 0.0026550511991463577, 0.006465530319528565, 0.005438591179846285, 0.0019995929370617274, 0.0028211688175613717, 0.004326631620445743, 0.006384098989903665, 0.004662070130834764, 0.00409398394806755, 0.0043272016869163425, 0.002676081299745038, 0.0030355618349172707, 0.005242632506146149, 0.007795296259096815, 0.006744671932791093, 0.0043051330493407945, 0.004696992944583231, 0.006671471073854059, 0.006765155310915872, 0.005376063739785014, 0.00606222046407843, 0.004324863001024804, 0.0042098570994180224, 0.0020440908759863175, 0.006061745683290165, 0.004652013920982763, 0.003067550556950907, 0.006333956940783763, 0.00408636326071897, 0.004343705445638353, 0.0025973536890256926, 0.004709533155743914, 0.004087695826223008, 0.00401387789402341, 0.005739330525633573, 0.004787162882611476, 0.006394011457518351, 0.004056081285830175, 0.0006482664956180162, 0.003104315280218495, 0.0006907787355834489, 0.002693423145293639, 0.0001, 0.004576872559339251, 0.0020268294838136918, 0.006109631731666516, 0.0006492311314251841, 0.00426532465572319, 0.0026969700756060144, 0.004794469557085987, 0.004485525597685074, 0.0002863346006671511, 0.005274279486979043, 0.004237714238682436, 0.004471460092701843, 0.006549082219106155, 0.0002905437157882096, 0.0010344187339995949, 0.004731235373319989, 0.006478884439009083, 0.000616131659244058, 0.0031454044168044613, 0.0008631898416422475, 0.0019695240202162465, 0.002599684984763269, 0.0020062337328834853, 0.0008630016285839993, 0.003090564313167305, 0.0031101742635007107, 0.0001, 0.0019784951656162102, 0.002663223735132855, 0.003232168321154265, 0.0030345556724274804, 0.0022857479382292077, 0.0036299134436701165, 0.0031167797237315904, 0.0019820694754630634, 0.002014550300789423, 0.004070630633565415, 0.00454807354590077, 0.005213306665677239, 0.0001, 0.005440389923333084, 0.0036315308923254405, 0.0066473896073971665, 0.005402417455214394, 0.004171362188972788, 0.0028789651442310375, 0.006620546441918804, 0.004355459159125599, 0.00411052376736605, 0.004000482864439394, 0.00417457455268706, 0.006901926694547949, 0.0006162591685356384, 0.002028627760199917, 0.002011818656479798, 0.0020413025563003253, 0.0020748566469324567, 0.0006490586406608579, 0.0031353663614596974, 0.0027276433304338115, 0.000652857256102558, 0.0001, 0.0001, 0.002684657945771365, 0.0030626182317151436, 0.0001, 0.004030344074106353, 0.004146756259624884, 0.007397986540492872, 0.00416959076404781, 0.006369751104024898, 0.004355860618041637, 0.00045194081831790396, 0.004736933699033137, 0.0020225010169944413, 0.0020159017802823796, 0.0022422002012688724, 0.00561018539752908, 0.0012362074566734228, 0.0020352162682292225, 0.002724170291963916, 0.0029621881620571795, 0.003580811964129191, 0.0020074424205067036, 0.0048099555109190235, 0.004322491227651624, 0.00038246825646780447, 0.0025334561876642157, 0.002042397087704864, 0.004390146710415098, 0.004958985094101863, 0.002025519786219654, 0.004007911566035813, 0.005163487172914655, 0.004036155718368452, 0.0020330848011643025, 0.0031094827286108086, 0.00407096920317374, 0.006602123010277424, 0.0043660465475951074, 0.0027149015129001846, 0.0020440208436436525, 0.004918051062770278, 0.004293033017448095, 0.0009153169134100396, 0.004727921191304524, 0.004029176791771982, 0.00283857671782892, 0.0001, 0.000647875212812456, 0.0007165222993730697, 0.006476011653809926, 0.002690850765296813, 0.007714784884510872, 0.004057450723387228, 0.00010386353967783219, 0.0020447215469693255, 0.004297618972742632, 0.0003312268414750422, 0.0041183593770054054, 0.006086274069369643, 0.005283880537939861, 0.00335862276923319, 0.004237465839054478, 0.003149476109968742, 0.002036820685727904, 0.0020394879398308538, 0.002045407217003381, 0.0036740195120143018, 0.003051993621264193, 0.002689660460081365, 0.002069791089260619, 0.0030668417988203543, 0.0026643904104600804, 0.004282284325379007, 0.005262336718135919, 0.0063929703139719825, 0.007523638180924408, 0.004003193608869681, 0.0001, 0.002713327086283403, 0.0020105890241559395, 0.0020006528874995395, 0.002888200938220527, 0.0041100776025071485, 0.002036700388965344, 0.0020450188995149526, 0.002040593405787012, 0.004062450134663692, 0.006285375076867792, 0.004081276921004397, 0.0020340603858105297, 0.002626066846885313, 0.0025765677012003396, 0.006322771191917969, 0.006022316588944107, 0.0001, 0.0027126690414923404, 0.0001, 0.0007270416037223542, 0.002125291393924622, 0.0006247154574194431, 0.002023799095871433, 0.00045795987861489847, 0.00029033826336547636, 0.0020424321638497374, 0.004057437992842191, 0.004034849895790447, 0.00028421030110406954, 0.0006663710181711001, 0.000293184685747717, 0.0001, 0.002082187061423569, 0.0001, 0.002600509420584395, 0.0020188588639835146, 0.0026208603049878505, 0.002018068046723442, 0.002938923746912601, 0.002024730394619318, 0.002033814069480922, 0.0036335030608892712, 0.002707702205042248, 0.0020180475269169535, 0.0034454658505866007, 0.001997146457086575, 0.005255720879130054, 0.004328908732737945, 0.002015328621976468, 0.0001, 0.0001, 0.002037132454080331, 0.0001, 0.004499527828737995, 0.005378952746402542, 0.004029877361570595, 0.0002846422681727653, 0.0008149091950958341, 0.004287936409529867, 0.0020175175111293727, 0.0020324335344145744, 0.005425059969212697, 0.005331866668339114, 0.0020767489577340494, 0.0047305906999226465, 0.006104030536373542, 0.0001, 0.0027308762971551717, 0.000286165270071663, 0.0006497806175429532, 0.0001, 0.0002848671872864204, 0.0006119772266113816, 0.0002896796724067028, 0.0001, 0.0020311853575263983, 0.002007450012514779, 0.00028719971537104265, 0.0019419256756003222, 0.0043284816983152575, 0.0006150652368741741, 0.004720156521246371, 0.006156090941793676, 0.0001, 0.002042405147785908, 0.002046186846604393, 0.0001, 0.0006949154161781357, 0.004753571425966649, 0.00422263397742226, 0.003307370991376194, 0.005548888749541389, 0.0020446824542754, 0.004171096446595869, 0.004050940663040943, 0.005842611725123738, 0.004061210290953265, 0.004347225227452584, 0.0020770374837955847, 0.0026864291707361875, 0.002656567141942692, 0.0020333628572751935, 0.0001, 0.0006167459020242971, 0.0020429173523766942, 0.0020032199960924816, 0.0006455631709855649, 0.00608553889637264, 0.0066958535404657024, 0.006085413975325809, 0.002016431052930991, 0.002012495281048352, 0.0033145365032146183, 0.0001, 0.0001, 0.0020041176460929215, 0.0026982290292564973, 0.00010289539310677784, 0.004814764504848629, 0.004055091895552971, 0.0020784837436774607, 0.0020157581421992483, 0.0020532350386021697, 0.0003007086049198535, 0.0001, 0.0026994755289700655, 0.0020416311448416516, 0.006384342656469755, 0.006596490115956728, 0.004317574764150027, 0.0001, 0.002013617750057795, 0.0020160200357197617, 0.003027096402610742, 0.00403522979626919, 0.004768258507102797, 0.0046127531041710804, 0.002012930416468615, 0.004061490550743394, 0.0020157837985873713, 0.0001, 0.004039219691859398, 0.0020451375544350203, 0.0001, 0.00030093645625933903, 0.00028913659374776325, 0.0032467677917834394, 0.0001, 0.0001, 0.0001, 0.0002861456163359808, 0.0002868434252762982, 0.0020453204288176795, 0.001948178297594273, 0.0040609632045111276, 0.006187983350429144, 0.0020435424265902945, 0.0006175689621702587, 0.0020257361467621235, 0.0033192375802504216, 0.0021759537789310117, 0.0020184594748231485, 0.0001, 0.002056951523800776, 0.0006125014882482892, 0.002048544337216525, 0.0001, 0.0034410417982061875, 0.005416559455175379, 0.0006527323856746388, 0.005373336042985564, 0.0001, 0.0001, 0.0001, 0.004061527900810724, 0.0001, 0.0001, 0.0020393052341195587, 0.0020164673886712667, 0.0002872501968065464, 0.0001, 0.002033554837374039, 0.0020130488668956273, 0.0001, 0.0020449324920685926, 0.0001, 0.004096487265084554, 0.004035375220093653, 0.0001, 0.0030053818755801003, 0.0026763485059122945, 0.002031437270462041, 0.0001, 0.0006191439134738283, 0.0020418288606660795, 0.0020412794383988164, 0.0020136737172413255, 0.0020152674662809195, 0.00472991150506711, 0.0040635469863890385, 0.0021736367543673265, 0.0019936812419204574, 0.0020161013528793357, 0.004747151253995484, 0.004061806554772514, 0.0001, 0.0002872367966674823, 0.0001, 0.0001, 0.0006962254947922897, 0.0001, 0.0020454704786059055, 0.0001, 0.0020168283448029787, 0.0001, 0.0001, 0.000287763247674368, 0.0001, 0.0002288506628121636, 0.002069088332754111, 0.0001, 0.0020162183941472784, 0.0001, 0.00028681871482736683, 0.00028750496835422824, 0.0001, 0.0001, 0.0020701490703833904, 0.0001, 0.0001, 0.0001, 0.0001, 0.002012072348703156, 0.002172591111633528, 0.0020149942274861328, 0.0001, 0.0001, 0.002674594909489946, 0.002067489417887997, 0.0001, 0.004330358181218375, 0.0001, 0.0001, 0.0001, 0.004061371019650237, 0.0076731604076530225, 0.004038697437104447, 0.0001, 0.00406085501220321, 0.004490511276638853, 0.0020449364266639037, 0.0040630735607530815, 0.004040747119280987, 0.0001, 0.0001, 0.0001, 0.0001, 0.0047476913515632855, 0.0001, 0.004061824882457354, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.000618884968538736, 0.0001, 0.0002876780921431773, 0.0001, 0.0001, 0.0001, 0.0006185666092665974, 0.0001, 0.002016032544693543, 0.0001, 0.0006188600047100982, 0.0001, 0.0001, 0.0020333292759317063, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0020158274936918725, 0.0001, 0.0003830011848646807, 0.002008785773033331, 0.0001, 0.002106576550350497, 0.004123706467361503, 0.0043839168988155175, 0.0006174078633011638, 0.0001, 0.0001, 0.0001, 0.002045464227654936, 0.0001, 0.0001, 0.0020448704103845495, 0.0001, 0.0006184168698712486, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0002874361124298778, 0.0001, 0.0001, 0.0001, 0.0020150048377219946, 0.0001, 0.0001, 0.0006187806237602177, 0.0001, 0.00028720226898293545, 0.0001, 0.0020614630400667393, 0.0001, 0.0001, 0.0001, 0.0020146152539477855, 0.0001, 0.0001, 0.0001, 0.0020743261450604346, 0.0020160581663781077, 0.0006176985294367974, 0.0020449999469627626, 0.0001, 0.0006316730241412536, 0.0001, 0.0020159280748682203, 0.0001, 0.0019268583507598923, 0.0001, 0.002016315446962673, 0.0001, 0.002044665502223847, 0.0001, 0.0001, 0.0001, 0.0001, 0.00028773443583455307, 0.00028767181949143264, 0.0001, 0.0001, 0.0001, 0.0001, 0.0020101151434112393, 0.0020519184163710316, 0.0001, 0.0006105388930974842, 0.0001, 0.0001, 0.0020575878117395506, 0.002042577682537954, 0.0001, 0.0003492235351704154, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0006531375714309915, 0.000621189818395449, 0.0001, 0.0001, 0.0020157524107865146, 0.0020101824230919704, 0.0001, 0.0001, 0.0020457839493275075, 0.0001, 0.0001, 0.0001, 0.0001, 0.0020335531345059567, 0.0001, 0.0001, 0.0001, 0.000620604467611702, 0.0001, 0.0001, 0.000285821099472496, 0.002013469881868212, 0.004803355261750662, 0.004729744456981881, 0.004061238905164245, 0.0020093661724538596, 0.0020165790315815253, 0.004071194120684567, 0.0001, 0.002032411317593804, 0.0004517871267893217, 0.0001, 0.0001, 0.0001, 0.00038355305162798804, 0.0001, 0.0006188119106331246, 0.002014609356437097, 0.0001, 0.0001, 0.002016301804600004, 0.002016214186879534, 0.0020160262920502986, 0.0006190629165712518, 0.0001, 0.002045144085651852, 0.002044751155592543, 0.0001, 0.00038371255856076223, 0.0020155282784752324, 0.0001, 0.0001, 0.002016057565552121, 0.0026531421630431955, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.00204477974598458, 0.0020462071563790794, 0.0001, 0.0001, 0.0001, 0.0020241796235068, 0.0001, 0.0001, 0.0020469786750793822, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0006188335570378279, 0.0006188713158015998, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.000616478761127552, 0.0001, 0.0001, 0.0001, 0.00028764059496088956, 0.0001, 0.0001, 0.0002992497953809639, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.002017560369188069, 0.0001, 0.002028711378272352, 0.0001, 0.0023324978568145154, 0.0001, 0.002015998178608875, 0.00022337147226602404, 0.0020449160634241465, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.002045075010645026, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0002859213452574923, 0.0001, 0.0040622030794625254, 0.004061063762092863, 0.0001, 0.0001, 0.002042153043554535, 0.0001, 0.0001, 0.0001, 0.0001, 0.0020448554594041726, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0009065087350195209, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0006163216190427797, 0.0001, 0.002015758897308604, 0.0001, 0.0001, 0.00201576494692494, 0.0020154099870029044, 0.002045056250320406, 0.0002877344480722984, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0020146018681919714, 0.0002890349832017874, 0.0001, 0.0001, 0.002040685597548168, 0.0020441850868833894, 0.002016551010779262, 0.0020114793587434066, 0.004059607388510325, 0.004061214385465487, 0.0001, 0.004061132568327042, 0.004337917014834455, 0.0002867080449821513, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0020175099012251347, 0.0001, 0.0001, 0.00204369377089028, 0.0001, 0.0001, 0.0001, 0.0001, 0.0006149897235171682, 0.0001, 0.00022817523076574704, 0.004061402162624922, 0.004058053783238019, 0.004181324344412703, 0.004060858028697994, 0.0001, 0.0004524746794279596, 0.0001, 0.0001, 0.0001, 0.0001, 0.0006191497018640456, 0.0001, 0.0001, 0.0006454737821908937, 0.0001, 0.0020160388076223557, 0.002016038809368959, 0.002004503893145282, 0.0001, 0.0023038494167930343, 0.0001, 0.002797499359870638, 0.002634434296605549, 0.0020446323035510972, 0.0001, 0.00028767810438331973, 0.000618968278609474, 0.0001, 0.0020510109732045623, 0.0020161951745493356, 0.002069475979813045, 0.0001, 0.00022775655289384314, 0.00028782820329504894, 0.002013643157023371, 0.0001, 0.00204488427507345, 0.0020161389007589884, 0.002015372511504357, 0.004745679019283297, 0.006060872723317467, 0.006054002301324584, 0.0017572781145519059, 0.004061201370707662, 0.0001, 0.0040611700987374655, 0.00205769128850434, 0.002687035999995766, 0.0001, 0.0001, 0.00028782192657434016, 0.0001, 0.0001, 0.0031091799909898203, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.00406110754757172, 0.004061070018317361, 0.0047456414373319485, 0.004061044998043731, 0.004061057506639089, 0.004061107548438778, 0.0001, 0.0001, 0.00022804380950158543, 0.0001, 0.0007249105355962836, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.00028775317219194573, 0.0001, 0.0001, 0.00028763427742019493, 0.0001, 0.0001, 0.0020159762516916346, 0.0001, 0.00022801886168326388, 0.0001, 0.0001, 0.002016594791914511, 0.0001, 0.0001, 0.0027268594558900296, 0.0001, 0.0006188306660652684, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0006188994935150825, 0.0001, 0.0001, 0.0001, 0.00028767181949143286, 0.0001, 0.0001, 0.0008077414609146844, 0.0001, 0.001495962901375229, 0.0001, 0.002004554211284908, 0.0001, 0.0020160513119385224, 0.0020160513171783, 0.0001, 0.0001, 0.0027271513473688383, 0.0001, 0.0001, 0.0001, 0.002016020035719761, 0.0001, 0.0001, 0.0020450312223970093, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0006188619788574218, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0002876280809392446, 0.0001, 0.0001, 0.0020160137791951447, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.002045068752803295, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0006187180822606428, 0.0001, 0.0001, 0.0001, 0.0002877156138110727, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.002016007525969711, 0.0001, 0.002044981183194508, 0.0001, 0.0001, 0.0001, 0.0001, 0.004061101291925271, 0.004061145076826088, 0.0001, 0.0001, 0.004061026231393378, 0.004061044997176658, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001]])
 
getting_e_field(time_points_r, electron_data, pulse_names, error1, [360, 355, 350, 345, 340, 335, 330, 325, 320, 315, 310, 305], 0.04, 'root_files/tcad_deplete_dope_0.0_-360.0.root')